In [18]:
import os
import pandas as pd
import numpy as np
import faiss
import torch
import time
from sentence_transformers import SentenceTransformer, models
from transformers import AutoTokenizer
import ast
import json
from typing import List, Dict, Tuple
import re


## Process


In [19]:
df = pd.read_csv('ingredients v1.csv')
df = df.drop(["Unnamed: 15","asins","sizes","weight","ean","upc","dateAdded","dateUpdated"], axis=1)  # Remove the unnamed column
df.info()

df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   id                  10000 non-null  object
 1   brand               9114 non-null   object
 2   categories          10000 non-null  object
 3   features.key        10000 non-null  object
 4   features.value      9972 non-null   object
 5   manufacturer        7313 non-null   object
 6   manufacturerNumber  6266 non-null   object
 7   name                9999 non-null   object
dtypes: object(8)
memory usage: 625.1+ KB


,id,brand,categories,features.key,features.value,manufacturer,manufacturerNumber,name
0,AVphBRHOilAPnD_x0OrE,Simon Fischer,"Grocery & Gourmet Food,Food,Grocery",Ingredients,"Dried Prunes,Water,Corn Syrup,Sugar,Pectin.",Sokol And Company,33829,Simon Fischer Fruit Bttr Prune Lekvar
1,AVpfNFy1LJeJML434ma2,McCormick,"Grocery & Gourmet Food,Food,Grocery",Ingredients,"Salt,Sugar,Molasses (Refinery Syrup, Molasses,...","McCormick & Co, Inc",MCLANE500373852,McCORMICK GRILL MATES MOLASSES BACON SEASONING...
2,AVpgT49VLJeJML43MJEz,Jolly Time,"Grocery & Gourmet Food,Grocery",Ingredients,"Salt, Yellow 5 Lake, Tricalcium Phosphate And ...",Reese's,NaN,Jolly Time Popcorn
3,AVphYgnzLJeJML43aPp2,Ziyad,"Grocery & Gourmet Food,grocery",Ingredients,Mechanically hulled seasame seeds.Allergy Info...,Ziyad,NaN,Ziyad Tahini Sesame Sauce
4,AVpiS0bOLJeJML43kRsh,Fla-Vor-Ice,"Grocery & Gourmet Food,grocery",Ingredients,FALSE,Fla-Vor-Ice,NaN,Fla-Vor-Ice Plus Giant Pops


In [20]:
df.dropna(inplace=True)  # Remove rows with any missing values
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5195 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   id                  5195 non-null   object
 1   brand               5195 non-null   object
 2   categories          5195 non-null   object
 3   features.key        5195 non-null   object
 4   features.value      5195 non-null   object
 5   manufacturer        5195 non-null   object
 6   manufacturerNumber  5195 non-null   object
 7   name                5195 non-null   object
dtypes: object(8)
memory usage: 365.3+ KB


In [21]:
# Lọc chỉ giữ các hàng có features.key = "Ingredients"
df = df[df['features.key'] == 'Ingredients']
# Kiểm tra kết quả
print(f"Số hàng sau khi lọc: {len(df)}")
df.rename(columns={'features.value': 'ingredients'}, inplace=True)
df.drop(columns=['features.key'], inplace=True)  # Remove the key column
df.info()
df.head()

Số hàng sau khi lọc: 5119
<class 'pandas.core.frame.DataFrame'>
Index: 5119 entries, 0 to 9999
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   id                  5119 non-null   object
 1   brand               5119 non-null   object
 2   categories          5119 non-null   object
 3   ingredients         5119 non-null   object
 4   manufacturer        5119 non-null   object
 5   manufacturerNumber  5119 non-null   object
 6   name                5119 non-null   object
dtypes: object(7)
memory usage: 319.9+ KB


,id,brand,categories,ingredients,manufacturer,manufacturerNumber,name
0,AVphBRHOilAPnD_x0OrE,Simon Fischer,"Grocery & Gourmet Food,Food,Grocery","Dried Prunes,Water,Corn Syrup,Sugar,Pectin.",Sokol And Company,33829,Simon Fischer Fruit Bttr Prune Lekvar
1,AVpfNFy1LJeJML434ma2,McCormick,"Grocery & Gourmet Food,Food,Grocery","Salt,Sugar,Molasses (Refinery Syrup, Molasses,...","McCormick & Co, Inc",MCLANE500373852,McCORMICK GRILL MATES MOLASSES BACON SEASONING...
5,AVpfiMykilAPnD_xdedK,Hero,"Food,Other Grocery,Grocery","Red Raspberries,Sugar,Glucose Syrup,Citric Aci...","HERO, INC.",B1080406602008,Hero Fruit Sprd Blk Currant-12 Oz -pack of 8
6,AVpgPmxs1cnluZ0-ypMt,Simply Asia,"Grocery & Gourmet Food,Grocery","Noodles: wheat flour,water,wheat gluten,modifi...",Simply Asia,900034971,Simply Asia Noodle Bowl Mandarin Orange -- 8.5 oz
7,AVphcTTBLJeJML43a9fO,EMERIL S,"Food,Fresh Food,Grocery","Wheat Flour,Soybean Oil,Salt,Dehydrated Garlic...","B&G Foods, Inc.",50909512,Italian Bread Crumbs


In [22]:
# Chuẩn hóa text
df['name'] = df['name'].str.strip()  # Xóa khoảng trắng đầu cuối
df['name'] = df['name'].str.title()  # Viết hoa chữ cái đầu
df['ingredients'] = df['ingredients'].str.lower()  # Chuyển thành chữ thường
df = df.head(500).reset_index(drop=True)
df['id'] = range(len(df))



In [23]:
df["id"] = df.index  # Tạo cột id bắt đầu từ 1
df["id"].reset_index(drop=True, inplace=True)
df.head()


,id,brand,categories,ingredients,manufacturer,manufacturerNumber,name
0,0,Simon Fischer,"Grocery & Gourmet Food,Food,Grocery","dried prunes,water,corn syrup,sugar,pectin.",Sokol And Company,33829,Simon Fischer Fruit Bttr Prune Lekvar
1,1,McCormick,"Grocery & Gourmet Food,Food,Grocery","salt,sugar,molasses (refinery syrup, molasses,...","McCormick & Co, Inc",MCLANE500373852,Mccormick Grill Mates Molasses Bacon Seasoning...
2,2,Hero,"Food,Other Grocery,Grocery","red raspberries,sugar,glucose syrup,citric aci...","HERO, INC.",B1080406602008,Hero Fruit Sprd Blk Currant-12 Oz -Pack Of 8
3,3,Simply Asia,"Grocery & Gourmet Food,Grocery","noodles: wheat flour,water,wheat gluten,modifi...",Simply Asia,900034971,Simply Asia Noodle Bowl Mandarin Orange -- 8.5 Oz
4,4,EMERIL S,"Food,Fresh Food,Grocery","wheat flour,soybean oil,salt,dehydrated garlic...","B&G Foods, Inc.",50909512,Italian Bread Crumbs


In [24]:
df.to_csv('cleaned_data.csv', index=False)

In [25]:
def clean_text(text):
    if pd.isnull(text) or text in ['nan', 'None']:
        return ''
    # Loại bỏ ký tự lạ & khoảng trắng dư
    text = re.sub(r'[\xa0\n\r\t]+', ' ', str(text))
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

# Clean từng trường (trừ brand và name)
for col in ['categories', 'ingredients', 'manufacturer', 'manufacturerNumber']:
    df[col] = df[col].apply(lambda x: clean_text(x))

# Clean brand và name nhưng giữ nguyên kiểu viết hoa/thường
df['brand'] = df['brand'].apply(lambda x: '' if pd.isnull(x) else str(x).strip())
df['name'] = df['name'].apply(lambda x: '' if pd.isnull(x) else str(x).strip())

df['text_corpus'] = (
    "This product is a " + df['name'] + " from the brand " + df['brand'] + ". "
    "It falls under the category of " + df['categories'].str.lower() + " and contains ingredients such as " + df['ingredients'].str.lower() + ". "
    "It is manufactured by " + df['manufacturer'].str.lower() + " (manufacturer code: " + df['manufacturerNumber'].str.lower() + ")."
)



In [26]:
df[['id', 'name', 'brand', 'text_corpus']].to_csv("product_metadata.csv", index=False)


## Embedding

In [27]:
# model_name = 'BAAI/bge-base-en-v1.5'


In [28]:
model_name = 'BAAI/bge-large-en-v1.5'
model = SentenceTransformer(model_name)

# Kiểm tra tokenizer max length
max_lenght = model.tokenizer.model_max_length
print("Max length:", max_lenght)


Max length: 512


In [29]:
batch_size = 32

In [30]:

embeddings = model.encode(
    df['text_corpus'].tolist(),
    batch_size=batch_size,
    show_progress_bar=True,
    normalize_embeddings=True,
    max_length=max_lenght,  # Giới hạn độ dài tối đa của chuỗi
    # convert_to_tensor=True,
    device='cuda' if torch.cuda.is_available() else 'cpu'
)


Batches:   0%|          | 0/16 [00:00<?, ?it/s]

/home/vinh/.local/lib/python3.9/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
Batches: 100%|██████████| 16/16 [00:04<00:00,  3.65it/s]


In [31]:
np.save('embeddings.npy', embeddings)

In [32]:

# Khởi tạo model và tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tokenize dữ liệu text_corpus để lấy độ dài
df['token_length'] = df['text_corpus'].apply(lambda x: len(tokenizer.tokenize(x)))

# Kiểm tra các thống kê
print("Max token length:", df['token_length'].max())
print("Min token length:", df['token_length'].min())
print("Average token length:", df['token_length'].mean())

# Optional: Kiểm tra số mẫu vượt quá giới hạn mặc định
print(f"Số mẫu vượt quá {max_lenght} token:", (df['token_length'] > max_lenght).sum())


Token indices sequence length is longer than the specified maximum sequence length for this model (550 > 512). Running this sequence through the model will result in indexing errors


Max token length: 550
Min token length: 46
Average token length: 119.5
Số mẫu vượt quá 512 token: 1


In [33]:
ids_exceeding_max_len = df[df['token_length'] > max_lenght]['id'].tolist()
print(ids_exceeding_max_len)

[174]


## Advanced Text Processing với Attention Pooling
Xử lý các text corpus vượt quá max_length bằng cách chia chunk và sử dụng attention pooling

In [34]:
import torch
import torch.nn as nn
from sentence_transformers import SentenceTransformer, models
from transformers import AutoTokenizer

def split_text_into_chunks(text, tokenizer, max_length):
    """
    Chia text thành các chunks với độ dài tối đa max_length tokens
    """
    tokens = tokenizer.tokenize(text)
    chunks = []
    
    if len(tokens) <= max_length:
        return [text]  # Không cần chia nếu text đã ngắn
    
    for i in range(0, len(tokens), max_length - 20):  # Overlap 20 tokens để giữ context
        chunk = tokens[i:i+max_length]
        chunk_text = tokenizer.convert_tokens_to_string(chunk)
        chunks.append(chunk_text)
    
    return chunks

class AttentionPooling(nn.Module):
    """
    Attention pooling layer để kết hợp embeddings từ nhiều chunks
    """
    def __init__(self, embed_dim):
        super().__init__()
        self.attention = nn.Sequential(
            nn.Linear(embed_dim, 128),
            nn.Tanh(),
            nn.Linear(128, 1)
        )

    def forward(self, embeddings):
        # embeddings: [batch_size, num_chunks, embed_dim]
        scores = self.attention(embeddings)  # [batch_size, num_chunks, 1]
        weights = torch.softmax(scores, dim=1)  # [batch_size, num_chunks, 1]
        weighted = embeddings * weights  # [batch_size, num_chunks, embed_dim]
        pooled = weighted.sum(dim=1)  # [batch_size, embed_dim]
        return pooled

def embed_text_with_attention(text, model, tokenizer, max_length, device):
    """
    Embed text với attention pooling cho text dài
    """
    # Chia chunk
    chunks = split_text_into_chunks(text, tokenizer, max_length)
    
    if len(chunks) == 1:
        # Text ngắn, embed bình thường với các tham số nhất quán
        return model.encode(
            text, 
            batch_size=batch_size,
            show_progress_bar=False,
            normalize_embeddings=True,
            max_length=max_length,
            device=device,
            convert_to_tensor=True
        )
    
    # Embed từng chunk
    chunk_embeddings = []
    for chunk in chunks:
        emb = model.encode(
            chunk, 
            batch_size=batch_size,
            show_progress_bar=False,
            normalize_embeddings=True,
            max_length=max_length,
            device=device,
            convert_to_tensor=True
        ).unsqueeze(0)  # [1, embed_dim]
        chunk_embeddings.append(emb)
    
    all_chunks = torch.cat(chunk_embeddings, dim=0).unsqueeze(0)  # [1, num_chunks, embed_dim]
    
    # Attention Pooling
    attn_pool = AttentionPooling(embed_dim=all_chunks.shape[-1]).to(device)
    pooled_embedding = attn_pool(all_chunks.to(device))  # [1, embed_dim]
    
    # Normalize final embedding
    pooled_embedding = torch.nn.functional.normalize(pooled_embedding, p=2, dim=1)
    
    return pooled_embedding.squeeze(0)

print("✅ Advanced text processing functions loaded successfully!")

✅ Advanced text processing functions loaded successfully!


In [35]:
def create_embeddings_with_attention_pooling(df, model, tokenizer, max_length=max_lenght, batch_size=batch_size):
    """
    Tạo embeddings với attention pooling cho các text vượt quá max_length
    """
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"🔄 Creating embeddings with attention pooling (device: {device})")
    
    # Phân tích độ dài text
    token_lengths = df['text_corpus'].apply(lambda x: len(tokenizer.tokenize(x)))
    long_texts = token_lengths > max_length
    
    print(f"📊 Text length analysis:")
    print(f"   • Total texts: {len(df)}")
    print(f"   • Long texts (>{max_length} tokens): {long_texts.sum()} ({long_texts.mean()*100:.1f}%)")
    print(f"   • Max token length: {token_lengths.max()}")
    print(f"   • Average token length: {token_lengths.mean():.1f}")
    
    embeddings_list = []
    
    # Xử lý từng text
    for idx, (_, row) in enumerate(df.iterrows()):
        text = row['text_corpus']
        text_length = token_lengths.iloc[idx]
        
        if text_length <= max_length:
            # Text ngắn - embed bình thường với các tham số giống embedding gốc
            embedding = model.encode(
                text,
                batch_size=batch_size,
                show_progress_bar=False,
                normalize_embeddings=True,
                max_length=max_length,
                device=device,
                convert_to_tensor=True
            )
        else:
            # Text dài - sử dụng attention pooling
            embedding = embed_text_with_attention(
                text, model, tokenizer, max_length, device
            )
        
        embeddings_list.append(embedding.detach().cpu().numpy())
        
        # Progress bar
        if (idx + 1) % 50 == 0:
            print(f"   Processed {idx + 1}/{len(df)} texts...")
    
    # Convert to numpy array
    embeddings = np.array(embeddings_list)
    print(f"✅ Embeddings created: shape {embeddings.shape}")
    
    return embeddings

# Tạo embeddings mới với attention pooling
print("🚀 Starting advanced embedding creation...")
embeddings_attention = create_embeddings_with_attention_pooling(
    df, model, tokenizer, max_length=max_lenght  
)

🚀 Starting advanced embedding creation...
🔄 Creating embeddings with attention pooling (device: cuda)
📊 Text length analysis:
   • Total texts: 500
   • Long texts (>512 tokens): 1 (0.2%)
   • Max token length: 550
   • Average token length: 119.5


/home/vinh/.local/lib/python3.9/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


   Processed 50/500 texts...
   Processed 100/500 texts...
   Processed 150/500 texts...
   Processed 200/500 texts...
   Processed 250/500 texts...
   Processed 300/500 texts...
   Processed 350/500 texts...
   Processed 400/500 texts...
   Processed 450/500 texts...
   Processed 500/500 texts...
✅ Embeddings created: shape (500, 1024)


In [36]:
# Lưu embeddings mới
np.save('embeddings_attention_{max_lenght}.npy', embeddings_attention)

# Tạo FAISS index mới với embeddings attention
print("🔄 Creating new FAISS index with attention embeddings...")

dimension = embeddings_attention.shape[1]
index_attention = faiss.IndexFlatIP(dimension)  # Inner Product = Cosine similarity
index_attention.add(embeddings_attention)

# Lưu index mới
faiss.write_index(index_attention, "faiss_index_attention_384.index")

print(f"✅ New FAISS index created with {index_attention.ntotal} vectors")
print(f"📁 Files saved:")
print(f"   • embeddings_attention_{max_lenght}.npy")
print(f"   • faiss_index_attention_{max_lenght}.index")

🔄 Creating new FAISS index with attention embeddings...
✅ New FAISS index created with 500 vectors
📁 Files saved:
   • embeddings_attention_512.npy
   • faiss_index_attention_512.index


In [37]:
def search_with_attention(query, top_k=3, max_length=max_lenght):
    """
    Search function sử dụng attention pooling cho long queries
    """
    # Load index attention
    index_attn = faiss.read_index("faiss_index_attention_384.index")
    
    # Encode query với attention pooling nếu cần
    time_start = time.time()
    
    query_length = len(tokenizer.tokenize(query))
    
    if query_length <= max_length:
        # Query ngắn - embed bình thường
        query_embedding = model.encode(
            [query],
            normalize_embeddings=True,
            device='cuda' if torch.cuda.is_available() else 'cpu'
        )
    else:
        # Query dài - sử dụng attention pooling
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        query_embedding = embed_text_with_attention(
            query, model, tokenizer, max_length, device
        ).unsqueeze(0).cpu().numpy()
    
    # Search trong FAISS index
    distances, indices = index_attn.search(query_embedding, top_k)
    time_end = time.time()
    response_time = (time_end - time_start) * 1000
    
    # Lấy metadata
    results = []
    for idx, score in zip(indices[0], distances[0]):
        if idx < len(metadata_df):
            row = metadata_df.iloc[idx]
            results.append({
                'id': row['id'],
                'name': row['name'],
                'brand': row['brand'],
                'score': float(score),
                'text': row['text_corpus'],
                'time': response_time,
                'method': 'attention_pooling'
            })
    
    return results

print("✅ Advanced search function with attention pooling created!")

✅ Advanced search function with attention pooling created!


In [38]:
embeddings = np.load("embeddings.npy")  # shape: (n_samples, 384)

In [39]:

# === Bước 1: Load embedding vectors từ file ===
embeddings = np.load("embeddings.npy")  # shape: (n_samples, 384)

# === Bước 2: Khởi tạo FAISS Index với cosine similarity ===
# Do embedding đã normalize → cosine = inner product
dimension = embeddings.shape[1]
index = faiss.IndexFlatIP(dimension)  # Inner Product = Cosine similarity nếu vectors đã chuẩn hóa

# === Bước 3: Thêm toàn bộ vectors vào index ===
index.add(embeddings)  # embeddings shape: (n_samples, dim)

# === Bước 4: Lưu FAISS index ra file để dùng lại sau ===
faiss.write_index(index, "faiss_index_cosine.index")

print(f"FAISS index created with {index.ntotal} vectors.")


FAISS index created with 500 vectors.


In [40]:
import faiss
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
import time

# === 1. Load FAISS index và metadata ===
index = faiss.read_index("faiss_index_cosine.index")
metadata_df = pd.read_csv("product_metadata.csv")



# === 3. Hàm truy vấn ===
def search(query, top_k=3):
    # Bước 1: Encode câu hỏi
    time_start = time.time()
    query_embedding = model.encode(
    [query],
    batch_size=batch_size,
    show_progress_bar=True,
    normalize_embeddings=True,
    max_length=max_lenght,  # Giới hạn độ dài tối đa của chuỗi
    device='cuda' if torch.cuda.is_available() else 'cpu'
)

    # Bước 2: Tìm top-k kết quả gần nhất
    distances, indices = index.search(query_embedding, top_k)
    time_end = time.time()
    response_time = (time_end - time_start) * 1000  # Thời gian tính bằng ms

    # Bước 3: Lấy thông tin từ metadata
    results = []
    for idx, score in zip(indices[0], distances[0]):
        if idx < len(metadata_df):
            row = metadata_df.iloc[idx]
            results.append({
                'id': row['id'],
                'name': row['name'],
                'brand': row['brand'],
                'score': float(score),
                'text': row['text_corpus'],
                'time': response_time 
                })
    return results

In [41]:
query = "List all items under the 'canned vegetables' category "
top_results = search(query, top_k=3)

for i, res in enumerate(top_results, 1):
    print(f"[{i}] {res['name']} + ({res['brand']})")
    print(f"→ Mức độ phù hợp: {res['score']:.4f}")
    print(f"→ Mô tả: {res['text']}\n")
    print(f"⏱️ Thời gian phản hồi: {res['time']:.2f} ms\n")
    print("id: ", res['id'])


Batches:   0%|          | 0/1 [00:00<?, ?it/s]/home/vinh/.local/lib/python3.9/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
Batches: 100%|██████████| 1/1 [00:00<00:00, 61.49it/s]

[1] Kitchen Basicsï¿½Ï¿½ Unsalted Vegetable Cooking Stock 8.25 Oz. Aseptic Carton + (Kitchen Basics)
→ Mức độ phù hợp: 0.6385
→ Mô tả: This product is a Kitchen Basicsï¿½Ï¿½ Unsalted Vegetable Cooking Stock 8.25 Oz. Aseptic Carton from the brand Kitchen Basics. It falls under the category of soups,food,grocery and contains ingredients such as vegetable stocks (onion, celery, carrot, mushroom, red pepper),tomato paste.. It is manufactured by kitchen basics,inc (manufacturer code: 1065424).

⏱️ Thời gian phản hồi: 29.24 ms

id:  219
[2] Kuner'Sï¿½Ï¿½ Southwest Sweet Corn & Peppers With Red & Green Peppers 15.25 Oz. Can + (Kuner's)
→ Mức độ phù hợp: 0.6080
→ Mô tả: This product is a Kuner'Sï¿½Ï¿½ Southwest Sweet Corn & Peppers With Red & Green Peppers 15.25 Oz. Can from the brand Kuner's. It falls under the category of food,canned vegetables,grocery and contains ingredients such as corn,water,red green peppers,salt.. It is manufactured by kuner-empson division of faribault foods, inc. (ma

## Evaluation System
Đánh giá hiệu suất hệ thống retrieval dựa trên ground truth

In [42]:
def calculate_hit_at_k(retrieved_ids: List[int], relevant_ids: List[int], k: int = 3) -> float:
    """
    Calculate Hit@K metric
    Returns 1.0 if any of the top-k results is relevant, 0.0 otherwise
    """
    top_k_ids = retrieved_ids[:k]
    return 1.0 if any(doc_id in relevant_ids for doc_id in top_k_ids) else 0.0

def calculate_mrr(retrieved_ids: List[int], relevant_ids: List[int]) -> float:
    """
    Calculate Mean Reciprocal Rank (MRR)
    Returns the reciprocal of the rank of the first relevant document
    """
    for rank, doc_id in enumerate(retrieved_ids, 1):
        if doc_id in relevant_ids:
            return 1.0 / rank
    return 0.0

def calculate_precision_at_k(retrieved_ids: List[int], relevant_ids: List[int], k: int = 3) -> float:
    """
    Calculate Precision@K with adaptive k
    If relevant_ids < k, then use len(relevant_ids) as k for fair evaluation
    Returns the proportion of relevant documents in top-k results
    """
    # Điều chỉnh k nếu số relevant documents ít hơn k
    effective_k = min(k, len(relevant_ids))
    
    if effective_k == 0:
        return 0.0
    
    top_k_ids = retrieved_ids[:effective_k]
    relevant_in_top_k = sum(1 for doc_id in top_k_ids if doc_id in relevant_ids)
    
    return relevant_in_top_k / effective_k

def evaluate_single_query(query: str, relevant_ids: List[int], search_func, k: int = 10) -> Dict:
    """
    Evaluate a single query and return metrics
    """
    # Perform search
    results = search_func(query, top_k=k)
    
    # Extract retrieved IDs and response time
    retrieved_ids = [int(res['id']) for res in results]
    response_time = results[0]['time'] if results else 0
    
    # Calculate metrics with adaptive precision
    hit_at_3 = calculate_hit_at_k(retrieved_ids, relevant_ids, k=3)
    mrr = calculate_mrr(retrieved_ids, relevant_ids)
    precision_at_3 = calculate_precision_at_k(retrieved_ids, relevant_ids, k=3)
    
    # Thêm thông tin về effective k cho precision
    effective_k_for_precision = min(3, len(relevant_ids))
    
    return {
        'query': query,
        'retrieved_ids': retrieved_ids,
        'relevant_ids': relevant_ids,
        'hit_at_3': hit_at_3,
        'mrr': mrr,
        'precision_at_3': precision_at_3,
        'effective_precision_k': effective_k_for_precision,  # Thêm thông tin này
        'response_time_ms': response_time,
        'num_relevant': len(relevant_ids),
        'num_retrieved': len(retrieved_ids)
    }

print("✅ Updated evaluation functions with adaptive Precision@K!")
print("📊 Now Precision@K uses k = min(K, num_ground_truth) for fair evaluation")

✅ Updated evaluation functions with adaptive Precision@K!
📊 Now Precision@K uses k = min(K, num_ground_truth) for fair evaluation


In [43]:


# Load ground truth data
gt_df = pd.read_csv("gt.csv")
print(f"Loaded {len(gt_df)} ground truth queries")

# Parse relevant_doc_ids from string to list
def parse_doc_ids(doc_ids_str):
    """Parse document IDs string to list of integers"""
    try:
        # Remove quotes and parse as list
        doc_ids_str = doc_ids_str.strip('"')
        return ast.literal_eval(doc_ids_str)
    except:
        # Fallback parsing method
        doc_ids_str = doc_ids_str.replace('[', '').replace(']', '').replace('"', '')
        return [int(x.strip()) for x in doc_ids_str.split(',') if x.strip().isdigit()]

gt_df['relevant_doc_ids'] = gt_df['relevant_doc_ids'].apply(parse_doc_ids)

# Display sample
print("\nSample ground truth data:")
for i in range(3):
    query = gt_df.iloc[i]['query']
    relevant_ids = gt_df.iloc[i]['relevant_doc_ids']
    print(f"Query: {query}")
    print(f"Relevant IDs: {relevant_ids[:5]}...")  # Show first 5 IDs
    print()

Loaded 22 ground truth queries

Sample ground truth data:
Query: Which products contain garlic powder of Utz brand?
Relevant IDs: [258, 276, 288, 308, 325]...

Query: Find all products by the brand Kikkoman
Relevant IDs: [7, 8, 289, 376]...

Query: Which products are from the brand Spice Islands?
Relevant IDs: [15, 33, 38, 49, 60]...



In [44]:
# Phân tích ground truth distribution
print("📊 GROUND TRUTH DISTRIBUTION ANALYSIS")
print("="*60)

# Đếm số relevant documents cho mỗi query
gt_distribution = gt_df['relevant_doc_ids'].apply(len)

print(f"📈 Statistics:")
print(f"   Total queries: {len(gt_df)}")
print(f"   Min relevant docs: {gt_distribution.min()}")
print(f"   Max relevant docs: {gt_distribution.max()}")
print(f"   Mean relevant docs: {gt_distribution.mean():.1f}")
print(f"   Median relevant docs: {gt_distribution.median():.1f}")

print(f"\n📋 Distribution by number of relevant documents:")
dist_counts = gt_distribution.value_counts().sort_index()
for num_docs, count in dist_counts.items():
    percentage = count / len(gt_df) * 100
    adaptive_k = min(3, num_docs)
    print(f"   {num_docs} relevant docs: {count} queries ({percentage:.1f}%) → Effective P@{adaptive_k}")

print(f"\n🎯 Impact on Precision@K evaluation:")
queries_with_fewer_than_3 = (gt_distribution < 3).sum()
print(f"   Queries with < 3 relevant docs: {queries_with_fewer_than_3}/{len(gt_df)} ({queries_with_fewer_than_3/len(gt_df)*100:.1f}%)")
print(f"   These queries will use adaptive P@K where K = number of relevant docs")

# Ví dụ một số queries có ít relevant documents
print(f"\n🔍 Examples of queries with few relevant documents:")
few_relevant_queries = gt_df[gt_distribution <= 2].head(3)
for idx, row in few_relevant_queries.iterrows():
    query = row['query']
    relevant_ids = row['relevant_doc_ids']
    print(f"   • Query: '{query[:50]}...'")
    print(f"     Relevant IDs: {relevant_ids} → Will use P@{len(relevant_ids)} instead of P@3")

📊 GROUND TRUTH DISTRIBUTION ANALYSIS
📈 Statistics:
   Total queries: 22
   Min relevant docs: 2
   Max relevant docs: 18
   Mean relevant docs: 8.4
   Median relevant docs: 9.0

📋 Distribution by number of relevant documents:
   2 relevant docs: 3 queries (13.6%) → Effective P@2
   4 relevant docs: 2 queries (9.1%) → Effective P@3
   5 relevant docs: 2 queries (9.1%) → Effective P@3
   6 relevant docs: 2 queries (9.1%) → Effective P@3
   7 relevant docs: 1 queries (4.5%) → Effective P@3
   8 relevant docs: 1 queries (4.5%) → Effective P@3
   10 relevant docs: 4 queries (18.2%) → Effective P@3
   11 relevant docs: 1 queries (4.5%) → Effective P@3
   12 relevant docs: 3 queries (13.6%) → Effective P@3
   14 relevant docs: 1 queries (4.5%) → Effective P@3
   15 relevant docs: 1 queries (4.5%) → Effective P@3
   18 relevant docs: 1 queries (4.5%) → Effective P@3

🎯 Impact on Precision@K evaluation:
   Queries with < 3 relevant docs: 3/22 (13.6%)
   These queries will use adaptive P@K where

In [45]:
def run_full_evaluation(gt_df: pd.DataFrame, search_func, k: int = 10) -> Dict:
    """
    Run evaluation on all ground truth queries
    """
    results = []
    
    print("Running evaluation on all queries...")
    for idx, row in gt_df.iterrows():
        query = row['query']
        relevant_ids = row['relevant_doc_ids']
        
        # Evaluate single query
        eval_result = evaluate_single_query(query, relevant_ids, search_func, k)
        results.append(eval_result)
        
        # Progress indicator
        if (idx + 1) % 5 == 0:
            print(f"Processed {idx + 1}/{len(gt_df)} queries")
    
    # Calculate overall metrics
    total_queries = len(results)
    overall_hit_at_3 = sum(r['hit_at_3'] for r in results) / total_queries * 100
    overall_mrr = sum(r['mrr'] for r in results) / total_queries * 100
    overall_precision_at_3 = sum(r['precision_at_3'] for r in results) / total_queries * 100
    avg_response_time = sum(r['response_time_ms'] for r in results) / total_queries
    
    # Response time statistics
    response_times = [r['response_time_ms'] for r in results]
    min_response_time = min(response_times)
    max_response_time = max(response_times)
    
    # Count queries meeting criteria
    hit_at_3_target = sum(1 for r in results if r['hit_at_3'] >= 1.0)
    mrr_target = sum(1 for r in results if r['mrr'] >= 0.5)
    response_time_target = sum(1 for r in results if r['response_time_ms'] <= 200)
    
    summary = {
        'total_queries': total_queries,
        'overall_metrics': {
            'hit_at_3_percent': overall_hit_at_3,
            'mrr_percent': overall_mrr,
            'precision_at_3_percent': overall_precision_at_3,
            'avg_response_time_ms': avg_response_time,
            'min_response_time_ms': min_response_time,
            'max_response_time_ms': max_response_time
        },
        'target_achievement': {
            'hit_at_3_100_percent': (hit_at_3_target / total_queries) * 100,
            'mrr_above_50_percent': (mrr_target / total_queries) * 100,
            'response_time_under_200ms': (response_time_target / total_queries) * 100
        },
        'detailed_results': results
    }
    
    return summary

# Run the evaluation
evaluation_results = run_full_evaluation(gt_df, search)
print("Evaluation completed!")

Running evaluation on all queries...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]/home/vinh/.local/lib/python3.9/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 68.77it/s]


Processed 5/22 queries


Batches: 100%|██████████| 1/1 [00:00<00:00, 82.12it/s]


...

Processed 15/22 queries


Batches: 100%|██████████| 1/1 [00:00<00:00, 87.98it/s]


Processed 20/22 queries


Batches: 100%|██████████| 1/1 [00:00<00:00, 98.99it/s]

Evaluation completed!


In [46]:
# Chạy lại evaluation với Adaptive Precision@K
print("🔄 Re-running evaluation with updated adaptive Precision@K...")
print("="*70)

# Update evaluation functions message
print("✅ Using Adaptive Precision@K where:")
print("   • P@K with K = min(3, number_of_ground_truth_docs)")
print("   • Fair evaluation for queries with few relevant documents")
print("   • More accurate performance measurement")

# Chạy lại tất cả evaluations
evaluation_results_adaptive = run_full_evaluation(gt_df, search)
evaluation_results_attention_adaptive = run_full_evaluation(gt_df, search_with_attention)

print("\n📊 ADAPTIVE PRECISION@K RESULTS:")
print("="*50)

# So sánh kết quả adaptive
adaptive_basic_metrics = evaluation_results_adaptive['overall_metrics']
adaptive_attention_metrics = evaluation_results_attention_adaptive['overall_metrics']

print(f"\n{'Method':<20} {'Hit@3':<10} {'MRR':<10} {'Adaptive P@K':<15} {'Avg Time':<12}")
print("-" * 70)
print(f"{'Basic Bi-Encoder':<20} {adaptive_basic_metrics['hit_at_3_percent']:<10.1f} {adaptive_basic_metrics['mrr_percent']:<10.1f} {adaptive_basic_metrics['precision_at_3_percent']:<15.1f} {adaptive_basic_metrics['avg_response_time_ms']:<12.1f}")
print(f"{'Attention Bi-Encoder':<20} {adaptive_attention_metrics['hit_at_3_percent']:<10.1f} {adaptive_attention_metrics['mrr_percent']:<10.1f} {adaptive_attention_metrics['precision_at_3_percent']:<15.1f} {adaptive_attention_metrics['avg_response_time_ms']:<12.1f}")

print("✅ Evaluation with Adaptive Precision@K completed!")

🔄 Re-running evaluation with updated adaptive Precision@K...
✅ Using Adaptive Precision@K where:
   • P@K with K = min(3, number_of_ground_truth_docs)
   • Fair evaluation for queries with few relevant documents
   • More accurate performance measurement
Running evaluation on all queries...


Batches: 100%|██████████| 1/1 [00:00<00:00, 82.05it/s]


Processed 5/22 queries


Batches: 100%|██████████| 1/1 [00:00<00:00, 86.93it/s]


Processed 10/22 queries


...

Processed 15/22 queries


Batches: 100%|██████████| 1/1 [00:00<00:00, 84.56it/s]


Processed 20/22 queries


Batches: 100%|██████████| 1/1 [00:00<00:00, 96.65it/s]


Running evaluation on all queries...
Processed 5/22 queries
Processed 10/22 queries
Processed 15/22 queries
Processed 20/22 queries

📊 ADAPTIVE PRECISION@K RESULTS:

Method               Hit@3      MRR        Adaptive P@K    Avg Time    
----------------------------------------------------------------------
Basic Bi-Encoder     90.9       79.1       65.2            18.8        
Attention Bi-Encoder 86.4       78.7       63.6            16.7        
✅ Evaluation with Adaptive Precision@K completed!


In [47]:
# So sánh performance giữa method cũ và mới
print("🔄 Running evaluation comparison...")

# Đánh giá với attention pooling
evaluation_results_attention = run_full_evaluation(gt_df, search_with_attention)

print("\n" + "="*80)
print("📊 COMPARISON: BASIC vs ATTENTION POOLING")
print("="*80)

# So sánh kết quả
basic_metrics = evaluation_results['overall_metrics']
attention_metrics = evaluation_results_attention['overall_metrics']

basic_targets = evaluation_results['target_achievement']
attention_targets = evaluation_results_attention['target_achievement']

print(f"\n{'Metric':<25} {'Basic':<15} {'Attention':<15} {'Improvement':<15}")
print("-" * 70)
print(f"{'Hit@3 (%)':<25} {basic_metrics['hit_at_3_percent']:<15.1f} {attention_metrics['hit_at_3_percent']:<15.1f} {attention_metrics['hit_at_3_percent'] - basic_metrics['hit_at_3_percent']:+.1f}")
print(f"{'MRR (%)':<25} {basic_metrics['mrr_percent']:<15.1f} {attention_metrics['mrr_percent']:<15.1f} {attention_metrics['mrr_percent'] - basic_metrics['mrr_percent']:+.1f}")
print(f"{'Precision@3 (%)':<25} {basic_metrics['precision_at_3_percent']:<15.1f} {attention_metrics['precision_at_3_percent']:<15.1f} {attention_metrics['precision_at_3_percent'] - basic_metrics['precision_at_3_percent']:+.1f}")
print(f"{'Avg Response (ms)':<25} {basic_metrics['avg_response_time_ms']:<15.1f} {attention_metrics['avg_response_time_ms']:<15.1f} {attention_metrics['avg_response_time_ms'] - basic_metrics['avg_response_time_ms']:+.1f}")

print(f"\n{'Target Achievement':<25} {'Basic':<15} {'Attention':<15} {'Improvement':<15}")
print("-" * 70)
print(f"{'Hit@3 = 100% (queries)':<25} {basic_targets['hit_at_3_100_percent']:<15.1f} {attention_targets['hit_at_3_100_percent']:<15.1f} {attention_targets['hit_at_3_100_percent'] - basic_targets['hit_at_3_100_percent']:+.1f}")
print(f"{'MRR > 50% (queries)':<25} {basic_targets['mrr_above_50_percent']:<15.1f} {attention_targets['mrr_above_50_percent']:<15.1f} {attention_targets['mrr_above_50_percent'] - basic_targets['mrr_above_50_percent']:+.1f}")
print(f"{'Time < 200ms (queries)':<25} {basic_targets['response_time_under_200ms']:<15.1f} {attention_targets['response_time_under_200ms']:<15.1f} {attention_targets['response_time_under_200ms'] - basic_targets['response_time_under_200ms']:+.1f}")

# Hiển thị cải thiện tổng thể
basic_targets_met = sum([
    basic_targets['hit_at_3_100_percent'] == 100,
    basic_targets['mrr_above_50_percent'] >= 50,
    basic_targets['response_time_under_200ms'] >= 90
])

attention_targets_met = sum([
    attention_targets['hit_at_3_100_percent'] == 100,
    attention_targets['mrr_above_50_percent'] >= 50,
    attention_targets['response_time_under_200ms'] >= 90
])

print(f"\n🎯 TARGETS MET:")
print(f"   Basic Method: {basic_targets_met}/3")
print(f"   Attention Method: {attention_targets_met}/3")

if attention_targets_met > basic_targets_met:
    print("🎉 ATTENTION POOLING IMPROVED PERFORMANCE!")
elif attention_targets_met == basic_targets_met:
    print("➡️  Same number of targets met, but check individual improvements")
else:
    print("⚠️  Need further optimization")

🔄 Running evaluation comparison...
Running evaluation on all queries...
Processed 5/22 queries
Processed 10/22 queries
Processed 15/22 queries
Processed 20/22 queries

📊 COMPARISON: BASIC vs ATTENTION POOLING

Metric                    Basic           Attention       Improvement    
----------------------------------------------------------------------
Hit@3 (%)                 90.9            86.4            -4.5
MRR (%)                   79.1            78.7            -0.4
Precision@3 (%)           65.2            63.6            -1.5
Avg Response (ms)         19.3            17.3            -2.0

Target Achievement        Basic           Attention       Improvement    
----------------------------------------------------------------------
Hit@3 = 100% (queries)    90.9            86.4            -4.5
MRR > 50% (queries)       77.3            77.3            +0.0
Time < 200ms (queries)    100.0           100.0           +0.0

🎯 TARGETS MET:
   Basic Method: 2/3
   Attention Method: 

In [48]:
def display_evaluation_results(eval_results: Dict):
    """
    Display evaluation results in a formatted way
    """
    print("="*60)
    print("🎯 RETRIEVAL SYSTEM EVALUATION RESULTS")
    print("="*60)
    
    # Overall metrics
    metrics = eval_results['overall_metrics']
    targets = eval_results['target_achievement']
    total_queries = eval_results['total_queries']
    
    print(f"\n📊 OVERALL PERFORMANCE ({total_queries} queries):")
    print("-"*40)
    print(f"Hit@3 (Average):          {metrics['hit_at_3_percent']:.1f}%")
    print(f"MRR (Average):            {metrics['mrr_percent']:.1f}%")
    print(f"Precision@3 (Average):    {metrics['precision_at_3_percent']:.1f}%")
    print(f"Avg Response Time:        {metrics['avg_response_time_ms']:.1f} ms")
    print(f"Response Time Range:      {metrics['min_response_time_ms']:.1f} - {metrics['max_response_time_ms']:.1f} ms")
    
    print(f"\n🎯 TARGET ACHIEVEMENT:")
    print("-"*40)
    
    # Hit@3 = 100% target
    hit_achievement = targets['hit_at_3_100_percent']
    hit_status = "✅ ACHIEVED" if hit_achievement == 100 else "❌ NOT ACHIEVED"
    print(f"Hit@3 = 100%:             {hit_achievement:.1f}% of queries {hit_status}")
    
    # MRR > 50% target
    mrr_achievement = targets['mrr_above_50_percent']
    mrr_status = "✅ ACHIEVED" if mrr_achievement > 50 else "❌ NOT ACHIEVED"
    print(f"MRR > 50%:                {mrr_achievement:.1f}% of queries {mrr_status}")
    
    # Response time < 200ms target
    time_achievement = targets['response_time_under_200ms']
    time_status = "✅ ACHIEVED" if time_achievement >= 90 else "❌ NOT ACHIEVED"  # 90% threshold
    print(f"Response Time < 200ms:    {time_achievement:.1f}% of queries {time_status}")
    
    print(f"\n📈 SUMMARY:")
    print("-"*40)
    targets_met = sum([
        hit_achievement == 100,
        mrr_achievement > 50,
        time_achievement >= 90
    ])
    print(f"Targets Met: {targets_met}/3")
    
    if targets_met == 3:
        print("🎉 ALL TARGETS ACHIEVED!")
    else:
        print("⚠️  Some targets need improvement")
        
    return eval_results

# Display results
display_evaluation_results(evaluation_results)

🎯 RETRIEVAL SYSTEM EVALUATION RESULTS

📊 OVERALL PERFORMANCE (22 queries):
----------------------------------------
Hit@3 (Average):          90.9%
MRR (Average):            79.1%
Precision@3 (Average):    65.2%
Avg Response Time:        19.3 ms
Response Time Range:      16.6 - 22.6 ms

🎯 TARGET ACHIEVEMENT:
----------------------------------------
Hit@3 = 100%:             90.9% of queries ❌ NOT ACHIEVED
MRR > 50%:                77.3% of queries ✅ ACHIEVED
Response Time < 200ms:    100.0% of queries ✅ ACHIEVED

📈 SUMMARY:
----------------------------------------
Targets Met: 2/3
⚠️  Some targets need improvement


{'total_queries': 22,
 'overall_metrics': {'hit_at_3_percent': 90.9090909090909,
  'mrr_percent': 79.0909090909091,
  'precision_at_3_percent': 65.15151515151516,
  'avg_response_time_ms': 19.31885155764493,
  'min_response_time_ms': 16.637563705444336,
  'max_response_time_ms': 22.63045310974121},
 'target_achievement': {'hit_at_3_100_percent': 90.9090909090909,
  'mrr_above_50_percent': 77.27272727272727,
  'response_time_under_200ms': 100.0},
 'detailed_results': [{'query': 'Which products contain garlic powder of Utz brand?',
   'retrieved_ids': [402, 288, 449, 265, 15, 308, 325, 425, 495, 276],
   'relevant_ids': [258, 276, 288, 308, 325, 372, 402, 404, 449, 495],
   'hit_at_3': 1.0,
   'mrr': 1.0,
   'precision_at_3': 1.0,
   'effective_precision_k': 3,
   'response_time_ms': 19.021034240722656,
   'num_relevant': 10,
   'num_retrieved': 10},
  {'query': 'Find all products by the brand Kikkoman',
   'retrieved_ids': [7, 8, 289, 376, 32, 91, 483, 365, 452, 368],
   'relevant_ids':

In [49]:
def analyze_failed_queries(eval_results: Dict, show_details: bool = True):
    """
    Analyze queries that failed to meet targets
    """
    results = eval_results['detailed_results']
    
    # Find problematic queries
    failed_hit_at_3 = [r for r in results if r['hit_at_3'] < 1.0]
    failed_mrr = [r for r in results if r['mrr'] < 0.5]
    slow_queries = [r for r in results if r['response_time_ms'] > 200]
    
    print("🔍 DETAILED ANALYSIS:")
    print("="*50)
    
    print(f"\n❌ Queries with Hit@3 < 100% ({len(failed_hit_at_3)} queries):")
    if failed_hit_at_3 and show_details:
        for i, result in enumerate(failed_hit_at_3[:5], 1):  # Show top 5
            print(f"\n[{i}] Query: {result['query'][:60]}...")
            print(f"    Hit@3: {result['hit_at_3']:.2f}, MRR: {result['mrr']:.3f}")
            print(f"    Retrieved IDs: {result['retrieved_ids'][:5]}")
            print(f"    Relevant IDs: {result['relevant_ids'][:5]}")
    
    print(f"\n❌ Queries with MRR < 50% ({len(failed_mrr)} queries):")
    if failed_mrr and show_details:
        for i, result in enumerate(failed_mrr[:5], 1):
            print(f"\n[{i}] Query: {result['query'][:60]}...")
            print(f"    MRR: {result['mrr']:.3f} ({result['mrr']*100:.1f}%)")
            print(f"    Retrieved IDs: {result['retrieved_ids'][:5]}")
            print(f"    Relevant IDs: {result['relevant_ids'][:5]}")
    
    print(f"\n⏱️ Slow queries (> 200ms) ({len(slow_queries)} queries):")
    if slow_queries and show_details:
        for i, result in enumerate(slow_queries[:5], 1):
            print(f"\n[{i}] Query: {result['query'][:60]}...")
            print(f"    Response Time: {result['response_time_ms']:.1f} ms")
    
    return {
        'failed_hit_at_3': failed_hit_at_3,
        'failed_mrr': failed_mrr,
        'slow_queries': slow_queries
    }

# Analyze problematic queries
analysis = analyze_failed_queries(evaluation_results, show_details=True)

🔍 DETAILED ANALYSIS:

❌ Queries with Hit@3 < 100% (2 queries):

[1] Query: Show me all items containing sauce...
    Hit@3: 0.00, MRR: 0.200
    Retrieved IDs: [416, 184, 222, 119, 329]
    Relevant IDs: [3, 28, 38, 45, 60]

[2] Query: Which products contain orange juice?...
    Hit@3: 0.00, MRR: 0.200
    Retrieved IDs: [453, 239, 339, 244, 253]
    Relevant IDs: [3, 168, 253, 259, 403]

❌ Queries with MRR < 50% (5 queries):

[1] Query: Show me all items containing sauce...
    MRR: 0.200 (20.0%)
    Retrieved IDs: [416, 184, 222, 119, 329]
    Relevant IDs: [3, 28, 38, 45, 60]

[2] Query: Find all products by the brand Polaner...
    MRR: 0.333 (33.3%)
    Retrieved IDs: [64, 48, 34, 27, 360]
    Relevant IDs: [27, 34]

[3] Query: Fresh food products contain carrageenan?...
    MRR: 0.333 (33.3%)
    Retrieved IDs: [141, 421, 13, 427, 17]
    Relevant IDs: [6, 7, 8, 13, 31]

[4] Query: Which products contain orange juice?...
    MRR: 0.200 (20.0%)
    Retrieved IDs: [453, 239, 339, 2

In [50]:
def create_evaluation_report(eval_results: Dict) -> pd.DataFrame:
    """
    Create a detailed evaluation report as DataFrame
    """
    results = eval_results['detailed_results']
    
    # Create detailed results DataFrame
    report_data = []
    for i, result in enumerate(results, 1):
        effective_k = result.get('effective_precision_k', 3)
        report_data.append({
            'Query_ID': i,
            'Query': result['query'][:80] + '...' if len(result['query']) > 80 else result['query'],
            'Hit@3': result['hit_at_3'],
            'MRR': result['mrr'],
            'Precision@3': result['precision_at_3'],
            'Effective_K': effective_k,  # Thêm cột này
            'Response_Time_ms': result['response_time_ms'],
            'Num_Relevant': result['num_relevant'],
            'Num_Retrieved': result['num_retrieved'],
            'Hit@3_Pass': '✅' if result['hit_at_3'] >= 1.0 else '❌',
            'MRR_Pass': '✅' if result['mrr'] >= 0.5 else '❌',
            'Time_Pass': '✅' if result['response_time_ms'] <= 200 else '❌'
        })
    
    report_df = pd.DataFrame(report_data)
    
    # Phân tích Effective K
    effective_k_stats = report_df['Effective_K'].value_counts().sort_index()
    
    # Summary statistics
    print("📋 EVALUATION REPORT SUMMARY:")
    print("="*50)
    print(f"Total Queries Evaluated: {len(report_df)}")
    
    print(f"\n📊 EFFECTIVE K DISTRIBUTION:")
    for k, count in effective_k_stats.items():
        percentage = count / len(report_df) * 100
        print(f"   K={k}: {count} queries ({percentage:.1f}%)")
    
    print(f"\nPASS RATES:")
    print(f"Hit@3 = 100%:     {(report_df['Hit@3'] >= 1.0).sum()}/{len(report_df)} ({(report_df['Hit@3'] >= 1.0).mean()*100:.1f}%)")
    print(f"MRR ≥ 50%:        {(report_df['MRR'] >= 0.5).sum()}/{len(report_df)} ({(report_df['MRR'] >= 0.5).mean()*100:.1f}%)")
    print(f"Time ≤ 200ms:     {(report_df['Response_Time_ms'] <= 200).sum()}/{len(report_df)} ({(report_df['Response_Time_ms'] <= 200).mean()*100:.1f}%)")
    
    print(f"\nMETRIC STATISTICS:")
    print(f"Hit@3:        Min: {report_df['Hit@3'].min():.2f}, Max: {report_df['Hit@3'].max():.2f}, Avg: {report_df['Hit@3'].mean():.2f}")
    print(f"MRR:          Min: {report_df['MRR'].min():.3f}, Max: {report_df['MRR'].max():.3f}, Avg: {report_df['MRR'].mean():.3f}")
    print(f"Precision@K:  Min: {report_df['Precision@3'].min():.3f}, Max: {report_df['Precision@3'].max():.3f}, Avg: {report_df['Precision@3'].mean():.3f}")
    print(f"Response Time: Min: {report_df['Response_Time_ms'].min():.1f}ms, Max: {report_df['Response_Time_ms'].max():.1f}ms, Avg: {report_df['Response_Time_ms'].mean():.1f}ms")
    
    return report_df

# Create and save evaluation report với updated function
evaluation_report_updated = create_evaluation_report(evaluation_results)

# Save to CSV
evaluation_report_updated.to_csv("evaluation_report_adaptive.csv", index=False)
print(f"\n💾 Updated evaluation report saved to 'evaluation_report_adaptive.csv'")

# Display first few rows with new column
print(f"\n📋 SAMPLE RESULTS (Top 10 queries) - WITH EFFECTIVE K:")
display_columns = ['Query_ID', 'Hit@3', 'MRR', 'Precision@3', 'Effective_K', 'Num_Relevant', 'Hit@3_Pass', 'MRR_Pass', 'Time_Pass']
print(evaluation_report_updated[display_columns].head(10))

📋 EVALUATION REPORT SUMMARY:
Total Queries Evaluated: 22

📊 EFFECTIVE K DISTRIBUTION:
   K=2: 3 queries (13.6%)
   K=3: 19 queries (86.4%)

PASS RATES:
Hit@3 = 100%:     20/22 (90.9%)
MRR ≥ 50%:        17/22 (77.3%)
Time ≤ 200ms:     22/22 (100.0%)

METRIC STATISTICS:
Hit@3:        Min: 0.00, Max: 1.00, Avg: 0.91
MRR:          Min: 0.200, Max: 1.000, Avg: 0.791
Precision@K:  Min: 0.000, Max: 1.000, Avg: 0.652
Response Time: Min: 16.6ms, Max: 22.6ms, Avg: 19.3ms

💾 Updated evaluation report saved to 'evaluation_report_adaptive.csv'

📋 SAMPLE RESULTS (Top 10 queries) - WITH EFFECTIVE K:
   Query_ID  Hit@3  MRR  Precision@3  Effective_K  Num_Relevant Hit@3_Pass  \
0         1    1.0  1.0     1.000000            3            10          ✅   
1         2    1.0  1.0     1.000000            3             4          ✅   
2         3    1.0  1.0     1.000000            3            10          ✅   
3         4    1.0  1.0     1.000000            3             4          ✅   
4         5    0.0

In [51]:
def display_failed_hit3_queries(eval_results: Dict):
    """
    Hiển thị chi tiết các query chưa đạt Hit@3 = 100%
    """
    results = eval_results['detailed_results']
    
    # Lọc các query có Hit@3 < 1.0 (chưa đạt 100%)
    failed_queries = [r for r in results if r['hit_at_3'] < 1.0]
    
    print("🔍 CHI TIẾT CÁC QUERY CHƯA ĐẠT HIT@3 = 100%")
    print("="*70)
    print(f"Tổng số query chưa đạt: {len(failed_queries)}/{len(results)} ({len(failed_queries)/len(results)*100:.1f}%)")
    print("="*70)
    
    if not failed_queries:
        print("🎉 Tất cả các query đều đạt Hit@3 = 100%!")
        return
    
    for i, result in enumerate(failed_queries, 1):
        print(f"\n🔸 QUERY #{i}")
        print("-" * 50)
        print(f"📝 Nội dung: {result['query']}")
        print(f"🎯 Hit@3: {result['hit_at_3']:.2f} ({result['hit_at_3']*100:.0f}%)")
        print(f"📊 MRR: {result['mrr']:.3f} ({result['mrr']*100:.1f}%)")
        print(f"📏 P@3: {result['precision_at_3']:.3f} ({result['precision_at_3']*100:.1f}%)")
        print(f"⏱️  Thời gian: {result['response_time_ms']:.1f} ms")
        
        print(f"\n📋 Kết quả truy vấn (Top 10):")
        for j, doc_id in enumerate(result['retrieved_ids'][:10], 1):
            is_relevant = "✅" if doc_id in result['relevant_ids'] else "❌"
            print(f"   {j}. ID {doc_id} {is_relevant}")
        
        print(f"\n🎯 Relevant IDs expected ({len(result['relevant_ids'])} docs):")
        relevant_str = ", ".join(map(str, result['relevant_ids'][:15]))
        if len(result['relevant_ids']) > 15:
            relevant_str += f", ... (+{len(result['relevant_ids'])-15} more)"
        print(f"   {relevant_str}")
        
        # Tính toán overlap cho Top-3 và Top-10
        retrieved_top3 = set(result['retrieved_ids'][:3])
        retrieved_top10 = set(result['retrieved_ids'][:10])
        relevant_set = set(result['relevant_ids'])
        overlap_3 = retrieved_top3.intersection(relevant_set)
        overlap_10 = retrieved_top10.intersection(relevant_set)
        
        print(f"\n📈 Phân tích:")
        print(f"   • Overlap trong Top-3: {len(overlap_3)} docs")
        print(f"   • Overlap trong Top-10: {len(overlap_10)} docs")
        print(f"   • Precision@3: {len(overlap_3)/3*100:.1f}%")
        print(f"   • Precision@10: {len(overlap_10)/10*100:.1f}%")
        print(f"   • Recall trong Top-3: {len(overlap_3)/len(relevant_set)*100:.1f}%")
        print(f"   • Recall trong Top-10: {len(overlap_10)/len(relevant_set)*100:.1f}%")
        
        if i < len(failed_queries):
            print("\n" + "="*70)
    
    return failed_queries

# Hiển thị các query chưa đạt Hit@3
failed_hit3_queries = display_failed_hit3_queries(evaluation_results)

🔍 CHI TIẾT CÁC QUERY CHƯA ĐẠT HIT@3 = 100%
Tổng số query chưa đạt: 2/22 (9.1%)

🔸 QUERY #1
--------------------------------------------------
📝 Nội dung: Show me all items containing sauce
🎯 Hit@3: 0.00 (0%)
📊 MRR: 0.200 (20.0%)
📏 P@3: 0.000 (0.0%)
⏱️  Thời gian: 22.3 ms

📋 Kết quả truy vấn (Top 10):
   1. ID 416 ❌
   2. ID 184 ❌
   3. ID 222 ❌
   4. ID 119 ❌
   5. ID 329 ✅
   6. ID 319 ❌
   7. ID 163 ❌
   8. ID 473 ❌
   9. ID 376 ✅
   10. ID 424 ❌

🎯 Relevant IDs expected (18 docs):
   3, 28, 38, 45, 60, 88, 107, 111, 114, 154, 174, 289, 329, 376, 405, ... (+3 more)

📈 Phân tích:
   • Overlap trong Top-3: 0 docs
   • Overlap trong Top-10: 2 docs
   • Precision@3: 0.0%
   • Precision@10: 20.0%
   • Recall trong Top-3: 0.0%
   • Recall trong Top-10: 11.1%


🔸 QUERY #2
--------------------------------------------------
📝 Nội dung: Which products contain orange juice?
🎯 Hit@3: 0.00 (0%)
📊 MRR: 0.200 (20.0%)
📏 P@3: 0.000 (0.0%)
⏱️  Thời gian: 18.6 ms

📋 Kết quả truy vấn (Top 10):
   1. ID 

In [52]:
def display_low_precision_queries(eval_results: Dict, precision_threshold: float = 0.33):
    """
    Hiển thị chi tiết các query có P@K thấp (dưới ngưỡng) với adaptive K
    """
    results = eval_results['detailed_results']
    
    # Lọc các query có P@K dưới ngưỡng
    low_precision_queries = [r for r in results if r['precision_at_3'] < precision_threshold]
    
    print(f"🔍 CHI TIẾT CÁC QUERY CÓ P@K < {precision_threshold*100:.0f}% (ADAPTIVE K)")
    print("="*70)
    print(f"Tổng số query có P@K thấp: {len(low_precision_queries)}/{len(results)} ({len(low_precision_queries)/len(results)*100:.1f}%)")
    print("="*70)
    
    if not low_precision_queries:
        print(f"🎉 Tất cả các query đều có P@K ≥ {precision_threshold*100:.0f}%!")
        return
    
    # Sắp xếp theo P@K tăng dần (worst first)
    low_precision_queries.sort(key=lambda x: x['precision_at_3'])
    
    for i, result in enumerate(low_precision_queries, 1):
        effective_k = result.get('effective_precision_k', min(3, len(result['relevant_ids'])))
        
        print(f"\n🔸 QUERY #{i}")
        print("-" * 50)
        print(f"📝 Nội dung: {result['query']}")
        print(f"📏 P@{effective_k}: {result['precision_at_3']:.3f} ({result['precision_at_3']*100:.1f}%)")
        print(f"🎯 Hit@3: {result['hit_at_3']:.2f} ({result['hit_at_3']*100:.0f}%)")
        print(f"📊 MRR: {result['mrr']:.3f} ({result['mrr']*100:.1f}%)")
        print(f"⏱️  Thời gian: {result['response_time_ms']:.1f} ms")
        print(f"🔢 Effective K: {effective_k} (Ground truth: {len(result['relevant_ids'])})")
        
        print(f"\n📋 Kết quả truy vấn Top-{effective_k}:")
        for j, doc_id in enumerate(result['retrieved_ids'][:effective_k], 1):
            is_relevant = "✅" if doc_id in result['relevant_ids'] else "❌"
            # Lấy tên sản phẩm nếu có
            product_name = ""
            if doc_id < len(metadata_df):
                product_name = f" - {metadata_df.iloc[doc_id]['name'][:50]}..."
            print(f"   {j}. ID {doc_id} {is_relevant}{product_name}")
        
        print(f"\n🎯 Relevant IDs expected ({len(result['relevant_ids'])} docs):")
        relevant_str = ", ".join(map(str, result['relevant_ids'][:10]))
        if len(result['relevant_ids']) > 10:
            relevant_str += f", ... (+{len(result['relevant_ids'])-10} more)"
        print(f"   {relevant_str}")
        
        # Phân tích chi tiết với effective k
        retrieved_top_k = set(result['retrieved_ids'][:effective_k])
        relevant_set = set(result['relevant_ids'])
        overlap_k = retrieved_top_k.intersection(relevant_set)
        
        print(f"\n📈 Phân tích chi tiết:")
        print(f"   • Relevant docs tìm được trong Top-{effective_k}: {len(overlap_k)}/{effective_k}")
        print(f"   • Tổng số relevant docs: {len(relevant_set)}")
        print(f"   • Precision@{effective_k}: {len(overlap_k)/effective_k*100:.1f}%")
        if effective_k <= len(relevant_set):
            max_possible = effective_k
        else:
            max_possible = len(relevant_set)
        print(f"   • Max possible precision@{effective_k}: {max_possible/effective_k*100:.1f}%")
        print(f"   • Recall@{effective_k}: {len(overlap_k)/len(relevant_set)*100:.1f}%")
        
        # Gợi ý cải thiện
        if len(overlap_k) == 0:
            print(f"   🚨 Không tìm được relevant doc nào trong Top-{effective_k}!")
        elif len(overlap_k) < effective_k:
            print(f"   ⚠️  Chỉ {len(overlap_k)}/{effective_k} kết quả relevant - cần cải thiện ranking")
        
        if i < len(low_precision_queries) and i < 10:  # Chỉ hiển thị tối đa 10 query
            print("\n" + "="*70)
    
    return low_precision_queries

# Phân tích các query có P@K thấp với adaptive K
print("\n" + "="*80)
low_precision_queries_adaptive = display_low_precision_queries(evaluation_results, precision_threshold=0.33)


🔍 CHI TIẾT CÁC QUERY CÓ P@K < 33% (ADAPTIVE K)
Tổng số query có P@K thấp: 3/22 (13.6%)

🔸 QUERY #1
--------------------------------------------------
📝 Nội dung: Show me all items containing sauce
📏 P@3: 0.000 (0.0%)
🎯 Hit@3: 0.00 (0%)
📊 MRR: 0.200 (20.0%)
⏱️  Thời gian: 22.3 ms
🔢 Effective K: 3 (Ground truth: 18)

📋 Kết quả truy vấn Top-3:
   1. ID 416 ❌ - Delallo Tomato Sauce, 15 Oz...
   2. ID 184 ❌ - Delallo Tomato Sauce, 8 Oz...
   3. ID 222 ❌ - Red Fork Sunday Pot Roast Seasoning Sauce, 8.0 Oz...

🎯 Relevant IDs expected (18 docs):
   3, 28, 38, 45, 60, 88, 107, 111, 114, 154, ... (+8 more)

📈 Phân tích chi tiết:
   • Relevant docs tìm được trong Top-3: 0/3
   • Tổng số relevant docs: 18
   • Precision@3: 0.0%
   • Max possible precision@3: 100.0%
   • Recall@3: 0.0%
   🚨 Không tìm được relevant doc nào trong Top-3!


🔸 QUERY #2
--------------------------------------------------
📝 Nội dung: Find all products by the brand Polaner
📏 P@2: 0.000 (0.0%)
🎯 Hit@3: 1.00 (100%)
📊 MRR: 0.

## Bi-Encoder + Cross-Encoder Approach
Kết hợp Bi-Encoder (để retrieval nhanh) và Cross-Encoder (để re-ranking chính xác)

In [53]:
from sentence_transformers import CrossEncoder
import torch

# Load Cross-Encoder model for re-ranking
print("🔄 Loading Cross-Encoder model...")
cross_encoder_model_name = 'BAAI/bge-reranker-base'  # Hoặc 'cross-encoder/ms-marco-MiniLM-L-6-v2'
cross_encoder = CrossEncoder(cross_encoder_model_name)

print(f"✅ Cross-Encoder loaded: {cross_encoder_model_name}")
print(f"   Device: {'cuda' if torch.cuda.is_available() else 'cpu'}")

# Kiểm tra device của cross-encoder
if torch.cuda.is_available():
    cross_encoder.model = cross_encoder.model.cuda()
    print("   Cross-Encoder moved to CUDA")

🔄 Loading Cross-Encoder model...
✅ Cross-Encoder loaded: BAAI/bge-reranker-base
   Device: cuda
   Cross-Encoder moved to CUDA


In [54]:
def hybrid_search_with_reranking(query, top_k=3, retrieval_k=20, use_attention=True):
    """
    Hybrid search với Bi-Encoder (retrieval) + Cross-Encoder (re-ranking)
    
    Args:
        query: Câu hỏi tìm kiếm
        top_k: Số lượng kết quả cuối cùng trả về
        retrieval_k: Số lượng candidates lấy từ Bi-Encoder (nên > top_k)
        use_attention: Sử dụng attention pooling hay không
    """
    time_start = time.time()
    
    # Bước 1: Bi-Encoder Retrieval (nhanh, lấy nhiều candidates)
    if use_attention:
        # Sử dụng attention pooling method
        bi_encoder_results = search_with_attention(query, top_k=retrieval_k)
    else:
        # Sử dụng basic method
        bi_encoder_results = search(query, top_k=retrieval_k)
    
    retrieval_time = time.time()
    
    # Bước 2: Chuẩn bị data cho Cross-Encoder
    query_doc_pairs = []
    candidate_docs = []
    
    for result in bi_encoder_results:
        # Tạo query-document pairs
        doc_text = result['text']
        query_doc_pairs.append([query, doc_text])
        candidate_docs.append(result)
    
    # Bước 3: Cross-Encoder Re-ranking (chậm nhưng chính xác)
    if len(query_doc_pairs) > 0:
        cross_encoder_scores = cross_encoder.predict(query_doc_pairs)
        
        # Gán scores mới cho candidates
        for i, doc in enumerate(candidate_docs):
            doc['cross_encoder_score'] = float(cross_encoder_scores[i])
            doc['bi_encoder_score'] = doc['score']  # Lưu lại score cũ
        
        # Sắp xếp lại theo Cross-Encoder scores
        candidate_docs.sort(key=lambda x: x['cross_encoder_score'], reverse=True)
    
    reranking_time = time.time()
    total_time = (reranking_time - time_start) * 1000
    
    # Bước 4: Trả về top-k kết quả
    final_results = candidate_docs[:top_k]
    for result in final_results:
        result['time'] = total_time
        result['method'] = f'hybrid_{"attention" if use_attention else "basic"}'
        result['retrieval_time_ms'] = (retrieval_time - time_start) * 1000
        result['reranking_time_ms'] = (reranking_time - retrieval_time) * 1000
    
    return final_results

print("✅ Hybrid search function with Bi-Encoder + Cross-Encoder created!")
print(f"   📊 Process: Bi-Encoder (fast retrieval) → Cross-Encoder (accurate re-ranking)")
print(f"   ⚡ Strategy: Retrieve {20} candidates, re-rank to top {3}")

✅ Hybrid search function with Bi-Encoder + Cross-Encoder created!
   📊 Process: Bi-Encoder (fast retrieval) → Cross-Encoder (accurate re-ranking)
   ⚡ Strategy: Retrieve 20 candidates, re-rank to top 3


In [55]:
# Test hybrid search function
test_query = "List all items under the 'canned vegetables' category"
print(f"🔍 Testing Hybrid Search: {test_query}")
print("="*70)

# So sánh 3 methods
methods = [
    ("Basic Bi-Encoder", lambda q: search(q, top_k=3)),
    ("Attention Bi-Encoder", lambda q: search_with_attention(q, top_k=3)),
    ("Hybrid + Basic", lambda q: hybrid_search_with_reranking(q, top_k=3, use_attention=False)),
    ("Hybrid + Attention", lambda q: hybrid_search_with_reranking(q, top_k=3, use_attention=True))
]

for method_name, search_func in methods:
    print(f"\n🔸 {method_name}:")
    print("-" * 40)
    
    try:
        results = search_func(test_query)
        
        for i, res in enumerate(results, 1):
            score_info = f"Score: {res['score']:.4f}"
            
            # Thêm thông tin Cross-Encoder score nếu có
            if 'cross_encoder_score' in res:
                score_info += f" | Cross-E: {res['cross_encoder_score']:.4f}"
            
            time_info = f"Time: {res['time']:.1f}ms"
            
            # Thêm thông tin breakdown time nếu có
            if 'retrieval_time_ms' in res and 'reranking_time_ms' in res:
                time_info += f" (Retrieval: {res['retrieval_time_ms']:.1f}ms, Rerank: {res['reranking_time_ms']:.1f}ms)"
            
            print(f"   [{i}] {res['name']} ({res['brand']})")
            print(f"       {score_info}")
            print(f"       {time_info}")
            
    except Exception as e:
        print(f"   ❌ Error: {str(e)}")
    
    print()

🔍 Testing Hybrid Search: List all items under the 'canned vegetables' category

🔸 Basic Bi-Encoder:
----------------------------------------


Batches:   0%|          | 0/1 [00:00<?, ?it/s]/home/vinh/.local/lib/python3.9/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
Batches: 100%|██████████| 1/1 [00:00<00:00, 53.04it/s]


   [1] Kitchen Basicsï¿½Ï¿½ Unsalted Vegetable Cooking Stock 8.25 Oz. Aseptic Carton (Kitchen Basics)
       Score: 0.6385
       Time: 31.0ms
   [2] Kuner'Sï¿½Ï¿½ Southwest Sweet Corn & Peppers With Red & Green Peppers 15.25 Oz. Can (Kuner's)
       Score: 0.6080
       Time: 31.0ms
   [3] Dell'Alpe Hot Giardiniera - 16Oz (Dell'Alpe)
       Score: 0.6063
       Time: 31.0ms


🔸 Attention Bi-Encoder:
----------------------------------------
   [1] Kitchen Basicsï¿½Ï¿½ Unsalted Vegetable Cooking Stock 8.25 Oz. Aseptic Carton (Kitchen Basics)
       Score: 0.6385
       Time: 105.4ms
   [2] Kuner'Sï¿½Ï¿½ Southwest Sweet Corn & Peppers With Red & Green Peppers 15.25 Oz. Can (Kuner's)
       Score: 0.6080
       Time: 105.4ms
   [3] Dell'Alpe Hot Giardiniera - 16Oz (Dell'Alpe)
       Score: 0.6063
       Time: 105.4ms


🔸 Hybrid + Basic:
----------------------------------------


Batches: 100%|██████████| 1/1 [00:00<00:00, 48.40it/s]
/home/vinh/.local/lib/python3.9/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


   [1] Diced Tomatoes (DeLallo)
       Score: 0.5958 | Cross-E: 0.9273
       Time: 147.1ms (Retrieval: 38.1ms, Rerank: 109.0ms)
   [2] Kidney Beans (Hanover)
       Score: 0.5908 | Cross-E: 0.9170
       Time: 147.1ms (Retrieval: 38.1ms, Rerank: 109.0ms)
   [3] Delallo Tomato Sauce, 15 Oz (DeLallo)
       Score: 0.5723 | Cross-E: 0.8687
       Time: 147.1ms (Retrieval: 38.1ms, Rerank: 109.0ms)


🔸 Hybrid + Attention:
----------------------------------------
   [1] Diced Tomatoes (DeLallo)
       Score: 0.5958 | Cross-E: 0.9273
       Time: 91.1ms (Retrieval: 18.9ms, Rerank: 72.2ms)
   [2] Kidney Beans (Hanover)
       Score: 0.5908 | Cross-E: 0.9170
       Time: 91.1ms (Retrieval: 18.9ms, Rerank: 72.2ms)
   [3] Delallo Tomato Sauce, 15 Oz (DeLallo)
       Score: 0.5723 | Cross-E: 0.8687
       Time: 91.1ms (Retrieval: 18.9ms, Rerank: 72.2ms)



In [56]:
# Chạy full evaluation với hybrid approaches
print("🚀 Running comprehensive evaluation with all methods...")
print("="*80)

# Tạo wrapper functions cho evaluation
def hybrid_basic_search(query, top_k=10):
    return hybrid_search_with_reranking(query, top_k=top_k, use_attention=False)

def hybrid_attention_search(query, top_k=10):
    return hybrid_search_with_reranking(query, top_k=top_k, use_attention=True)

# Chạy evaluation cho tất cả methods
evaluation_methods = [
    ("Basic Bi-Encoder", search),
    ("Attention Bi-Encoder", search_with_attention),
    ("Hybrid + Basic", hybrid_basic_search),
    ("Hybrid + Attention", hybrid_attention_search)
]

all_evaluations = {}

for method_name, search_func in evaluation_methods:
    print(f"\n🔄 Evaluating: {method_name}")
    print("-" * 50)
    
    try:
        eval_results = run_full_evaluation(gt_df, search_func)
        all_evaluations[method_name] = eval_results
        
        # Hiển thị kết quả ngắn gọn
        metrics = eval_results['overall_metrics']
        targets = eval_results['target_achievement']
        
        print(f"   Hit@3: {metrics['hit_at_3_percent']:.1f}%")
        print(f"   MRR: {metrics['mrr_percent']:.1f}%")
        print(f"   P@3: {metrics['precision_at_3_percent']:.1f}%")
        print(f"   Avg Time: {metrics['avg_response_time_ms']:.1f}ms")
        print(f"   Targets Met: {sum([targets['hit_at_3_100_percent']==100, targets['mrr_above_50_percent']>=50, targets['response_time_under_200ms']>=90])}/3")
        
    except Exception as e:
        print(f"   ❌ Error during evaluation: {str(e)}")
        continue

print("\n✅ All evaluations completed!")

🚀 Running comprehensive evaluation with all methods...

🔄 Evaluating: Basic Bi-Encoder
--------------------------------------------------
Running evaluation on all queries...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 45.71it/s]


Processed 5/22 queries


Batches: 100%|██████████| 1/1 [00:00<00:00, 80.63it/s]


...

Processed 15/22 queries


Batches: 100%|██████████| 1/1 [00:00<00:00, 86.90it/s]


Processed 20/22 queries


Batches: 100%|██████████| 1/1 [00:00<00:00, 94.76it/s]


   Hit@3: 100.0%
   MRR: 94.7%
   P@3: 83.3%
   Avg Time: 128.3ms
   Targets Met: 3/3

🔄 Evaluating: Hybrid + Attention
--------------------------------------------------
Running evaluation on all queries...
Processed 5/22 queries
Processed 10/22 queries
Processed 15/22 queries
Processed 20/22 queries
   Hit@3: 100.0%
   MRR: 94.7%
   P@3: 83.3%
   Avg Time: 154.9ms
   Targets Met: 2/3

✅ All evaluations completed!


In [57]:
# Chạy lại Hybrid evaluation với Adaptive Precision@K
print("🔄 Re-running Hybrid evaluation with Adaptive Precision@K...")
print("="*80)

# Chạy evaluation cho tất cả methods với adaptive P@K
all_evaluations_adaptive = {}

for method_name, search_func in evaluation_methods:
    print(f"\n🔄 Evaluating (Adaptive P@K): {method_name}")
    print("-" * 50)
    
    try:
        eval_results = run_full_evaluation(gt_df, search_func)
        all_evaluations_adaptive[method_name] = eval_results
        
        # Hiển thị kết quả ngắn gọn
        metrics = eval_results['overall_metrics']
        targets = eval_results['target_achievement']
        
        print(f"   Hit@3: {metrics['hit_at_3_percent']:.1f}%")
        print(f"   MRR: {metrics['mrr_percent']:.1f}%")
        print(f"   Adaptive P@K: {metrics['precision_at_3_percent']:.1f}%")
        print(f"   Avg Time: {metrics['avg_response_time_ms']:.1f}ms")
        print(f"   Targets Met: {sum([targets['hit_at_3_100_percent']==100, targets['mrr_above_50_percent']>=50, targets['response_time_under_200ms']>=90])}/3")
        
    except Exception as e:
        print(f"   ❌ Error during evaluation: {str(e)}")
        continue

print("\n✅ All Hybrid evaluations with Adaptive P@K completed!")

🔄 Re-running Hybrid evaluation with Adaptive Precision@K...

🔄 Evaluating (Adaptive P@K): Basic Bi-Encoder
--------------------------------------------------
Running evaluation on all queries...


Batches: 100%|██████████| 1/1 [00:00<00:00, 77.86it/s]


Processed 5/22 queries


Batches: 100%|██████████| 1/1 [00:00<00:00, 75.56it/s]


Processed 10/22 queries


...

Processed 15/22 queries


Batches: 100%|██████████| 1/1 [00:00<00:00, 69.32it/s]


Processed 20/22 queries


Batches: 100%|██████████| 1/1 [00:00<00:00, 93.17it/s]


   Hit@3: 100.0%
   MRR: 94.7%
   Adaptive P@K: 83.3%
   Avg Time: 132.2ms
   Targets Met: 3/3

🔄 Evaluating (Adaptive P@K): Hybrid + Attention
--------------------------------------------------
Running evaluation on all queries...
Processed 5/22 queries
Processed 10/22 queries
Processed 15/22 queries
Processed 20/22 queries
   Hit@3: 100.0%
   MRR: 94.7%
   Adaptive P@K: 83.3%
   Avg Time: 166.5ms
   Targets Met: 2/3

✅ All Hybrid evaluations with Adaptive P@K completed!


In [58]:
# So sánh kết quả cuối cùng: Fixed P@3 vs Adaptive P@K
def compare_fixed_vs_adaptive_precision(fixed_results, adaptive_results):
    """
    So sánh kết quả giữa Fixed P@3 và Adaptive P@K
    """
    print("📊 COMPARISON: FIXED P@3 vs ADAPTIVE P@K")
    print("="*80)
    
    if not fixed_results or not adaptive_results:
        print("❌ Missing evaluation results!")
        return
    
    print(f"\n{'Method':<20} {'Fixed P@3':<12} {'Adaptive P@K':<15} {'Difference':<12} {'Better':<8}")
    print("-" * 80)
    
    for method_name in fixed_results.keys():
        if method_name in adaptive_results:
            fixed_p3 = fixed_results[method_name]['overall_metrics']['precision_at_3_percent']
            adaptive_pk = adaptive_results[method_name]['overall_metrics']['precision_at_3_percent']
            difference = adaptive_pk - fixed_p3
            better = "📈" if difference > 0 else "📉" if difference < 0 else "➡️"
            
            print(f"{method_name:<20} {fixed_p3:<12.1f} {adaptive_pk:<15.1f} {difference:<12.1f} {better:<8}")
    
    print(f"\n🎯 WHY ADAPTIVE P@K IS MORE FAIR:")
    print("   • Fixed P@3: Penalizes queries with <3 relevant docs unfairly")
    print("   • Adaptive P@K: Uses K = min(3, num_ground_truth) for fair evaluation")
    print("   • Better reflects true system performance across all query types")
    
    # Tìm method tốt nhất với adaptive P@K
    best_adaptive_scores = {}
    for method_name, results in adaptive_results.items():
        metrics = results['overall_metrics']
        score = (
            metrics['hit_at_3_percent'] * 0.4 +
            metrics['mrr_percent'] * 0.3 +
            metrics['precision_at_3_percent'] * 0.3
        )
        best_adaptive_scores[method_name] = score
    
    best_method = max(best_adaptive_scores, key=best_adaptive_scores.get)
    print(f"\n🏆 BEST METHOD (Adaptive P@K): {best_method}")
    print(f"   Overall Score: {best_adaptive_scores[best_method]:.1f}")
    
    return adaptive_results

# Chạy so sánh nếu có cả 2 kết quả
if 'all_evaluations' in locals() and 'all_evaluations_adaptive' in locals():
    final_comparison = compare_fixed_vs_adaptive_precision(all_evaluations, all_evaluations_adaptive)
else:
    print("⚠️  Run both fixed and adaptive evaluations first to compare results.")

📊 COMPARISON: FIXED P@3 vs ADAPTIVE P@K

Method               Fixed P@3    Adaptive P@K    Difference   Better  
--------------------------------------------------------------------------------
Basic Bi-Encoder     65.2         65.2            0.0          ➡️      
Attention Bi-Encoder 63.6         63.6            0.0          ➡️      
Hybrid + Basic       83.3         83.3            0.0          ➡️      
Hybrid + Attention   83.3         83.3            0.0          ➡️      

🎯 WHY ADAPTIVE P@K IS MORE FAIR:
   • Fixed P@3: Penalizes queries with <3 relevant docs unfairly
   • Adaptive P@K: Uses K = min(3, num_ground_truth) for fair evaluation
   • Better reflects true system performance across all query types

🏆 BEST METHOD (Adaptive P@K): Hybrid + Basic
   Overall Score: 93.4


In [59]:
def compare_all_methods(evaluations_dict):
    """
    So sánh performance của tất cả methods
    """
    print("📊 COMPREHENSIVE PERFORMANCE COMPARISON")
    print("="*80)
    
    if not evaluations_dict:
        print("❌ No evaluation results to compare!")
        return
    
    # Tạo comparison table
    methods = list(evaluations_dict.keys())
    metrics_names = ['Hit@3 (%)', 'MRR (%)', 'P@3 (%)', 'Avg Time (ms)']
    
    print(f"\n{'Method':<20} {'Hit@3':<10} {'MRR':<10} {'P@3':<10} {'Time':<12} {'Targets':<10}")
    print("-" * 80)
    
    best_metrics = {'hit_at_3': 0, 'mrr': 0, 'precision_at_3': 0, 'time': float('inf')}
    best_methods = {'hit_at_3': '', 'mrr': '', 'precision_at_3': '', 'time': ''}
    
    for method_name in methods:
        eval_result = evaluations_dict[method_name]
        metrics = eval_result['overall_metrics']
        targets = eval_result['target_achievement']
        
        # Đếm targets đạt được
        targets_met = sum([
            targets['hit_at_3_100_percent'] == 100,
            targets['mrr_above_50_percent'] >= 50,
            targets['response_time_under_200ms'] >= 90
        ])
        
        # Cập nhật best metrics
        if metrics['hit_at_3_percent'] > best_metrics['hit_at_3']:
            best_metrics['hit_at_3'] = metrics['hit_at_3_percent']
            best_methods['hit_at_3'] = method_name
            
        if metrics['mrr_percent'] > best_metrics['mrr']:
            best_metrics['mrr'] = metrics['mrr_percent']
            best_methods['mrr'] = method_name
            
        if metrics['precision_at_3_percent'] > best_metrics['precision_at_3']:
            best_metrics['precision_at_3'] = metrics['precision_at_3_percent']
            best_methods['precision_at_3'] = method_name
            
        if metrics['avg_response_time_ms'] < best_metrics['time']:
            best_metrics['time'] = metrics['avg_response_time_ms']
            best_methods['time'] = method_name
        
        print(f"{method_name:<20} {metrics['hit_at_3_percent']:<10.1f} {metrics['mrr_percent']:<10.1f} {metrics['precision_at_3_percent']:<10.1f} {metrics['avg_response_time_ms']:<12.1f} {targets_met:<10}/3")
    
    print("\n🏆 BEST PERFORMANCE:")
    print("-" * 50)
    print(f"Best Hit@3:     {best_methods['hit_at_3']} ({best_metrics['hit_at_3']:.1f}%)")
    print(f"Best MRR:       {best_methods['mrr']} ({best_metrics['mrr']:.1f}%)")
    print(f"Best P@3:       {best_methods['precision_at_3']} ({best_metrics['precision_at_3']:.1f}%)")
    print(f"Fastest:        {best_methods['time']} ({best_metrics['time']:.1f}ms)")
    
    # Tìm method tốt nhất overall
    overall_scores = {}
    for method_name in methods:
        eval_result = evaluations_dict[method_name]
        metrics = eval_result['overall_metrics']
        targets = eval_result['target_achievement']
        
        # Tính overall score (có thể điều chỉnh weights)
        score = (
            metrics['hit_at_3_percent'] * 0.4 +  # 40% weight cho Hit@3
            metrics['mrr_percent'] * 0.3 +       # 30% weight cho MRR
            metrics['precision_at_3_percent'] * 0.3  # 30% weight cho P@3
        )
        
        # Penalty cho slow response (nếu > 200ms)
        if metrics['avg_response_time_ms'] > 200:
            score *= 0.9  # 10% penalty
            
        overall_scores[method_name] = score
    
    best_overall = max(overall_scores, key=overall_scores.get)
    print(f"\n🥇 BEST OVERALL METHOD: {best_overall}")
    print(f"   Overall Score: {overall_scores[best_overall]:.1f}")
    
    return evaluations_dict

# So sánh tất cả methods
if 'all_evaluations' in locals() and all_evaluations:
    comparison_results = compare_all_methods(all_evaluations)
else:
    print("⚠️  No evaluation results found. Please run the evaluation cells first.")

📊 COMPREHENSIVE PERFORMANCE COMPARISON

Method               Hit@3      MRR        P@3        Time         Targets   
--------------------------------------------------------------------------------
Basic Bi-Encoder     90.9       79.1       65.2       21.2         2         /3
Attention Bi-Encoder 86.4       78.7       63.6       14.9         2         /3
Hybrid + Basic       100.0      94.7       83.3       128.3        3         /3
Hybrid + Attention   100.0      94.7       83.3       154.9        2         /3

🏆 BEST PERFORMANCE:
--------------------------------------------------
Best Hit@3:     Hybrid + Basic (100.0%)
Best MRR:       Hybrid + Basic (94.7%)
Best P@3:       Hybrid + Basic (83.3%)
Fastest:        Attention Bi-Encoder (14.9ms)

🥇 BEST OVERALL METHOD: Hybrid + Basic
   Overall Score: 93.4


In [60]:
# Tối ưu hóa tham số cho Hybrid approach
def optimize_hybrid_parameters():
    """
    Tối ưu hóa retrieval_k parameter cho hybrid search
    """
    print("🔧 OPTIMIZING HYBRID SEARCH PARAMETERS")
    print("="*60)
    
    # Test với các giá trị retrieval_k khác nhau
    retrieval_k_values = [10, 15, 20, 30, 50]
    
    # Chọn một subset nhỏ của queries để test nhanh
    test_queries = gt_df.head(10)  # Test với 10 queries đầu tiên
    
    best_config = {'retrieval_k': 20, 'score': 0, 'use_attention': True}
    
    for use_attention in [False, True]:
        attention_str = "Attention" if use_attention else "Basic"
        print(f"\n🔸 Testing with {attention_str} Bi-Encoder:")
        print("-" * 40)
        
        for retrieval_k in retrieval_k_values:
            print(f"   Testing retrieval_k = {retrieval_k}...", end=" ")
            
            try:
                # Tạo search function với tham số cụ thể
                def test_search_func(query, top_k=10):
                    return hybrid_search_with_reranking(
                        query, 
                        top_k=top_k, 
                        retrieval_k=retrieval_k,
                        use_attention=use_attention
                    )
                
                # Evaluate với subset nhỏ
                test_results = run_full_evaluation(test_queries, test_search_func)
                metrics = test_results['overall_metrics']
                
                # Tính combined score
                combined_score = (
                    metrics['hit_at_3_percent'] * 0.4 +
                    metrics['mrr_percent'] * 0.3 +
                    metrics['precision_at_3_percent'] * 0.3
                )
                
                print(f"Score: {combined_score:.1f} (H@3: {metrics['hit_at_3_percent']:.1f}%, MRR: {metrics['mrr_percent']:.1f}%, Time: {metrics['avg_response_time_ms']:.0f}ms)")
                
                # Cập nhật best config
                if combined_score > best_config['score']:
                    best_config.update({
                        'retrieval_k': retrieval_k,
                        'score': combined_score,
                        'use_attention': use_attention,
                        'metrics': metrics
                    })
                    
            except Exception as e:
                print(f"Error: {str(e)}")
    
    print(f"\n🏆 BEST CONFIGURATION:")
    print("-" * 30)
    print(f"Retrieval K: {best_config['retrieval_k']}")
    print(f"Use Attention: {best_config['use_attention']}")
    print(f"Combined Score: {best_config['score']:.1f}")
    if 'metrics' in best_config:
        m = best_config['metrics']
        print(f"Hit@3: {m['hit_at_3_percent']:.1f}%")
        print(f"MRR: {m['mrr_percent']:.1f}%")
        print(f"P@3: {m['precision_at_3_percent']:.1f}%")
        print(f"Avg Time: {m['avg_response_time_ms']:.1f}ms")
    
    return best_config

# Chạy optimization (có thể bỏ qua nếu muốn tiết kiệm thời gian)
print("⚠️  Parameter optimization takes time. Set run_optimization = True to enable.")
run_optimization = False  # Set to True để chạy optimization

if run_optimization:
    optimal_config = optimize_hybrid_parameters()
else:
    print("Skipping parameter optimization. Using default: retrieval_k=20, use_attention=True")

⚠️  Parameter optimization takes time. Set run_optimization = True to enable.
Skipping parameter optimization. Using default: retrieval_k=20, use_attention=True


In [61]:
# Cập nhật các hàm display để sử dụng adaptive metrics nhất quán
def updated_display_metrics_summary(eval_results: Dict, method_name: str = ""):
    """
    Hiển thị tóm tắt metrics với adaptive precision thông tin rõ ràng
    """
    results = eval_results['detailed_results']
    metrics = eval_results['overall_metrics']
    
    print(f"\n📊 METRICS SUMMARY - {method_name}")
    print("="*60)
    
    # Basic metrics
    print(f"📈 Overall Performance:")
    print(f"   Hit@3: {metrics['hit_at_3_percent']:.1f}%")
    print(f"   MRR: {metrics['mrr_percent']:.1f}%")
    print(f"   Adaptive P@K: {metrics['precision_at_3_percent']:.1f}%")
    print(f"   Avg Response Time: {metrics['avg_response_time_ms']:.1f}ms")
    
    # Adaptive P@K breakdown
    print(f"\n🔍 Adaptive P@K Breakdown:")
    effective_k_counts = {}
    for result in results:
        effective_k = result.get('effective_precision_k', min(3, len(result['relevant_ids'])))
        if effective_k not in effective_k_counts:
            effective_k_counts[effective_k] = []
        effective_k_counts[effective_k].append(result['precision_at_3'])
    
    for k in sorted(effective_k_counts.keys()):
        queries_with_k = effective_k_counts[k]
        avg_precision_k = sum(queries_with_k) / len(queries_with_k) * 100
        print(f"   P@{k}: {len(queries_with_k)} queries, avg {avg_precision_k:.1f}%")
    
    # Target achievement
    targets = eval_results['target_achievement']
    print(f"\n🎯 Target Achievement:")
    print(f"   Hit@3 ≥ 95%: {targets['hit_at_3_100_percent']:.1f}% queries")
    print(f"   MRR ≥ 50%: {targets['mrr_above_50_percent']:.1f}% queries") 
    print(f"   Time ≤ 200ms: {targets['response_time_under_200ms']:.1f}% queries")

def updated_compare_methods_with_adaptive_metrics(evaluations_dict):
    """
    So sánh các methods với adaptive metrics được highlight rõ ràng
    """
    print("📊 COMPREHENSIVE METHOD COMPARISON WITH ADAPTIVE METRICS")
    print("="*80)
    
    # Header
    print(f"{'Method':<25} {'Hit@3':<10} {'MRR':<10} {'Adaptive P@K':<15} {'Time (ms)':<12} {'Targets':<10}")
    print("-" * 80)
    
    # Method comparison
    method_scores = {}
    for method_name, eval_results in evaluations_dict.items():
        metrics = eval_results['overall_metrics']
        targets = eval_results['target_achievement']
        
        # Count targets met
        targets_met = sum([
            targets['hit_at_3_100_percent'] >= 95,
            targets['mrr_above_50_percent'] >= 50, 
            targets['response_time_under_200ms'] >= 90
        ])
        
        # Display row
        print(f"{method_name:<25} {metrics['hit_at_3_percent']:<10.1f} {metrics['mrr_percent']:<10.1f} {metrics['precision_at_3_percent']:<15.1f} {metrics['avg_response_time_ms']:<12.1f} {targets_met}/3")
        
        # Calculate combined score for ranking
        combined_score = (
            metrics['hit_at_3_percent'] * 0.4 +
            metrics['mrr_percent'] * 0.3 +
            metrics['precision_at_3_percent'] * 0.3
        )
        method_scores[method_name] = combined_score
    
    # Best method
    best_method = max(method_scores.keys(), key=lambda x: method_scores[x])
    print(f"\n🏆 BEST METHOD (Adaptive Metrics): {best_method}")
    print(f"   Combined Score: {method_scores[best_method]:.1f}")
    
    # Adaptive P@K explanation
    print(f"\n💡 ADAPTIVE P@K EXPLANATION:")
    print(f"   • P@K uses K = min(3, num_ground_truth_docs)")
    print(f"   • Fair evaluation for queries with <3 relevant documents")
    print(f"   • More accurate performance measurement than fixed P@3")

print("✅ Updated display functions for consistent adaptive metrics reporting!")
print("🔧 Functions: updated_display_metrics_summary(), updated_compare_methods_with_adaptive_metrics()")

✅ Updated display functions for consistent adaptive metrics reporting!
🔧 Functions: updated_display_metrics_summary(), updated_compare_methods_with_adaptive_metrics()


In [62]:
# Cập nhật hàm analyze_failed_queries với adaptive metrics
def updated_analyze_failed_queries(eval_results: Dict, show_details: bool = True):
    """
    Analyze queries that failed to meet targets with adaptive precision info
    """
    results = eval_results['detailed_results']
    
    # Find problematic queries với adaptive metrics
    failed_hit_at_3 = [r for r in results if r['hit_at_3'] < 1.0]
    failed_mrr = [r for r in results if r['mrr'] < 0.5]
    slow_queries = [r for r in results if r['response_time_ms'] > 200]
    
    # Phân tích adaptive precision
    adaptive_precision_analysis = {}
    for result in results:
        effective_k = result.get('effective_precision_k', min(3, len(result['relevant_ids'])))
        if effective_k not in adaptive_precision_analysis:
            adaptive_precision_analysis[effective_k] = {
                'queries': [], 
                'avg_precision': 0,
                'pass_rate': 0
            }
        adaptive_precision_analysis[effective_k]['queries'].append(result)
    
    # Tính toán statistics cho mỗi adaptive K
    for k in adaptive_precision_analysis:
        queries = adaptive_precision_analysis[k]['queries']
        precisions = [q['precision_at_3'] for q in queries]
        adaptive_precision_analysis[k]['avg_precision'] = sum(precisions) / len(precisions)
        adaptive_precision_analysis[k]['pass_rate'] = sum(1 for p in precisions if p >= 0.7) / len(precisions)
    
    print("🔍 DETAILED ANALYSIS WITH ADAPTIVE METRICS")
    print("="*60)
    
    print(f"\n📊 Adaptive P@K Performance by K:")
    for k in sorted(adaptive_precision_analysis.keys()):
        analysis = adaptive_precision_analysis[k]
        print(f"   P@{k}: {len(analysis['queries'])} queries, avg {analysis['avg_precision']*100:.1f}%, pass rate {analysis['pass_rate']*100:.1f}%")
    
    print(f"\n❌ Queries with Hit@3 < 100% ({len(failed_hit_at_3)} queries):")
    if failed_hit_at_3 and show_details:
        for i, result in enumerate(failed_hit_at_3[:3], 1):  # Show top 3
            effective_k = result.get('effective_precision_k', min(3, len(result['relevant_ids'])))
            print(f"\n[{i}] Query: {result['query'][:60]}...")
            print(f"    Hit@3: {result['hit_at_3']:.2f}, MRR: {result['mrr']:.3f}")
            print(f"    Adaptive P@{effective_k}: {result['precision_at_3']:.3f} ({result['precision_at_3']*100:.1f}%)")
            print(f"    Retrieved IDs: {result['retrieved_ids'][:5]}")
            print(f"    Relevant IDs: {result['relevant_ids'][:5]}")
    
    print(f"\n❌ Queries with MRR < 50% ({len(failed_mrr)} queries):")
    if failed_mrr and show_details:
        for i, result in enumerate(failed_mrr[:3], 1):
            effective_k = result.get('effective_precision_k', min(3, len(result['relevant_ids'])))
            print(f"\n[{i}] Query: {result['query'][:60]}...")
            print(f"    MRR: {result['mrr']:.3f} ({result['mrr']*100:.1f}%)")
            print(f"    Adaptive P@{effective_k}: {result['precision_at_3']:.3f} ({result['precision_at_3']*100:.1f}%)")
            print(f"    Retrieved IDs: {result['retrieved_ids'][:5]}")
            print(f"    Relevant IDs: {result['relevant_ids'][:5]}")
    
    print(f"\n⏱️ Slow queries (> 200ms) ({len(slow_queries)} queries):")
    if slow_queries and show_details:
        for i, result in enumerate(slow_queries[:3], 1):
            effective_k = result.get('effective_precision_k', min(3, len(result['relevant_ids'])))
            print(f"\n[{i}] Query: {result['query'][:60]}...")
            print(f"    Response Time: {result['response_time_ms']:.1f} ms")
            print(f"    Adaptive P@{effective_k}: {result['precision_at_3']:.3f} ({result['precision_at_3']*100:.1f}%)")
            
    return {
        'failed_hit_at_3': failed_hit_at_3,
        'failed_mrr': failed_mrr,
        'slow_queries': slow_queries,
        'adaptive_precision_analysis': adaptive_precision_analysis
    }

print("✅ Updated analyze_failed_queries function with adaptive metrics support!")
print("🔧 Function: updated_analyze_failed_queries()")

✅ Updated analyze_failed_queries function with adaptive metrics support!
🔧 Function: updated_analyze_failed_queries()


In [63]:
# Hàm tổng hợp để chạy evaluation với adaptive metrics
def run_comprehensive_adaptive_evaluation():
    """
    Chạy evaluation toàn diện với adaptive metrics cho tất cả methods
    """
    print("🚀 COMPREHENSIVE ADAPTIVE METRICS EVALUATION")
    print("="*80)
    
    # Define all evaluation methods
    evaluation_methods = [
        ("Basic Bi-Encoder", search),
        ("Attention Bi-Encoder", search_with_attention),
        ("Hybrid + Basic", lambda query, top_k=10: hybrid_search_with_reranking(query, top_k=top_k, use_attention=False)),
        ("Hybrid + Attention", lambda query, top_k=10: hybrid_search_with_reranking(query, top_k=top_k, use_attention=True))
    ]
    
    all_evaluations_comprehensive = {}
    
    print(f"📊 Evaluating {len(evaluation_methods)} methods with adaptive P@K...")
    print(f"   • P@K uses K = min(3, num_ground_truth_documents)")
    print(f"   • Fair evaluation for all query types")
    print(f"   • Comprehensive analysis included")
    
    for method_name, search_func in evaluation_methods:
        print(f"\n🔄 Evaluating: {method_name}")
        print("-" * 50)
        
        try:
            # Run evaluation
            eval_results = run_full_evaluation(gt_df, search_func)
            all_evaluations_comprehensive[method_name] = eval_results
            
            # Display summary with adaptive metrics
            updated_display_metrics_summary(eval_results, method_name)
            
        except Exception as e:
            print(f"   ❌ Error during evaluation: {str(e)}")
            continue
    
    print(f"\n" + "="*80)
    print("📈 FINAL COMPARISON WITH ADAPTIVE METRICS")
    print(f"="*80)
    
    # Compare all methods
    updated_compare_methods_with_adaptive_metrics(all_evaluations_comprehensive)
    
    # Detailed analysis for best performing method
    if all_evaluations_comprehensive:
        # Find best method by combined score
        method_scores = {}
        for method_name, eval_results in all_evaluations_comprehensive.items():
            metrics = eval_results['overall_metrics']
            combined_score = (
                metrics['hit_at_3_percent'] * 0.4 +
                metrics['mrr_percent'] * 0.3 +
                metrics['precision_at_3_percent'] * 0.3
            )
            method_scores[method_name] = combined_score
        
        best_method = max(method_scores.keys(), key=lambda x: method_scores[x])
        
        print(f"\n🏆 DETAILED ANALYSIS OF BEST METHOD: {best_method}")
        print("="*80)
        updated_analyze_failed_queries(all_evaluations_comprehensive[best_method], show_details=True)
    
    print(f"\n✅ Comprehensive adaptive metrics evaluation completed!")
    return all_evaluations_comprehensive

# Test the new functions with existing data
if 'all_evaluations_adaptive' in locals() and all_evaluations_adaptive:
    print("🧪 Testing updated functions with existing adaptive evaluation data...")
    
    # Test updated display function
    sample_method = list(all_evaluations_adaptive.keys())[0]
    sample_results = all_evaluations_adaptive[sample_method]
    updated_display_metrics_summary(sample_results, sample_method)
    
    # Test updated comparison function  
    updated_compare_methods_with_adaptive_metrics(all_evaluations_adaptive)
    
    print("\n🎉 All updated functions working correctly with adaptive metrics!")
else:
    print("⚠️  Run the comprehensive evaluation to test all updated functions:")
    print("   final_evaluations = run_comprehensive_adaptive_evaluation()")

🧪 Testing updated functions with existing adaptive evaluation data...

📊 METRICS SUMMARY - Basic Bi-Encoder
📈 Overall Performance:
   Hit@3: 90.9%
   MRR: 79.1%
   Adaptive P@K: 65.2%
   Avg Response Time: 21.4ms

🔍 Adaptive P@K Breakdown:
   P@2: 3 queries, avg 66.7%
   P@3: 19 queries, avg 64.9%

🎯 Target Achievement:
   Hit@3 ≥ 95%: 90.9% queries
   MRR ≥ 50%: 77.3% queries
   Time ≤ 200ms: 100.0% queries
📊 COMPREHENSIVE METHOD COMPARISON WITH ADAPTIVE METRICS
Method                    Hit@3      MRR        Adaptive P@K    Time (ms)    Targets   
--------------------------------------------------------------------------------
Basic Bi-Encoder          90.9       79.1       65.2            21.4         2/3
Attention Bi-Encoder      86.4       78.7       63.6            16.0         2/3
Hybrid + Basic            100.0      94.7       83.3            132.2        3/3
Hybrid + Attention        100.0      94.7       83.3            166.5        2/3

🏆 BEST METHOD (Adaptive Metrics): Hybr

In [64]:
# Cập nhật hàm create_evaluation_report với adaptive metrics
def updated_create_evaluation_report(eval_results: Dict, method_name: str = "") -> pd.DataFrame:
    """
    Create a detailed evaluation report as DataFrame with adaptive metrics highlighted
    """
    results = eval_results['detailed_results']
    
    # Create detailed results DataFrame với adaptive metrics
    report_data = []
    for i, result in enumerate(results, 1):
        effective_k = result.get('effective_precision_k', min(3, len(result['relevant_ids'])))
        
        report_data.append({
            'Query_ID': i,
            'Query': result['query'][:50] + "..." if len(result['query']) > 50 else result['query'],
            'Hit@3': result['hit_at_3'],
            'MRR': result['mrr'], 
            'Precision@3_Fixed': result['precision_at_3'],  # Fixed P@3 for comparison
            'Effective_K': effective_k,
            f'Adaptive_P@K': result['precision_at_3'],  # Same value but labeled as adaptive
            'Response_Time_ms': result['response_time_ms'],
            'Num_Relevant_Docs': len(result['relevant_ids']),
            'Num_Retrieved_Docs': len(result['retrieved_ids']),
            'Hit@3_Pass': '✅' if result['hit_at_3'] >= 1.0 else '❌',
            'MRR_Pass': '✅' if result['mrr'] >= 0.5 else '❌',
            'Time_Pass': '✅' if result['response_time_ms'] <= 200 else '❌',
            'Adaptive_Precision_Pass': '✅' if result['precision_at_3'] >= 0.7 else '❌'
        })
    
    report_df = pd.DataFrame(report_data)
    
    # Adaptive K statistics
    effective_k_stats = report_df['Effective_K'].value_counts().sort_index()
    
    # Summary statistics với adaptive focus
    print(f"📋 ADAPTIVE EVALUATION REPORT SUMMARY - {method_name}")
    print("="*70)
    print(f"Total Queries Evaluated: {len(report_df)}")
    
    print(f"\n📊 ADAPTIVE P@K DISTRIBUTION:")
    for k, count in effective_k_stats.items():
        percentage = count / len(report_df) * 100
        avg_precision_k = report_df[report_df['Effective_K'] == k]['Adaptive_P@K'].mean() * 100
        print(f"   P@{k}: {count} queries ({percentage:.1f}%) - Avg: {avg_precision_k:.1f}%")
    
    print(f"\n📈 PASS RATES (Adaptive Metrics):")
    print(f"Hit@3 = 100%:      {(report_df['Hit@3'] >= 1.0).sum()}/{len(report_df)} ({(report_df['Hit@3'] >= 1.0).mean()*100:.1f}%)")
    print(f"MRR ≥ 50%:         {(report_df['MRR'] >= 0.5).sum()}/{len(report_df)} ({(report_df['MRR'] >= 0.5).mean()*100:.1f}%)")
    print(f"Adaptive P@K ≥ 70%: {(report_df['Adaptive_P@K'] >= 0.7).sum()}/{len(report_df)} ({(report_df['Adaptive_P@K'] >= 0.7).mean()*100:.1f}%)")
    print(f"Time ≤ 200ms:      {(report_df['Response_Time_ms'] <= 200).sum()}/{len(report_df)} ({(report_df['Response_Time_ms'] <= 200).mean()*100:.1f}%)")
    
    print(f"\n📊 METRIC STATISTICS (Adaptive):")
    print(f"Hit@3:         Min: {report_df['Hit@3'].min():.2f}, Max: {report_df['Hit@3'].max():.2f}, Avg: {report_df['Hit@3'].mean():.2f}")
    print(f"MRR:           Min: {report_df['MRR'].min():.3f}, Max: {report_df['MRR'].max():.3f}, Avg: {report_df['MRR'].mean():.3f}")
    print(f"Adaptive P@K:  Min: {report_df['Adaptive_P@K'].min():.3f}, Max: {report_df['Adaptive_P@K'].max():.3f}, Avg: {report_df['Adaptive_P@K'].mean():.3f}")
    print(f"Response Time: Min: {report_df['Response_Time_ms'].min():.1f}ms, Max: {report_df['Response_Time_ms'].max():.1f}ms, Avg: {report_df['Response_Time_ms'].mean():.1f}ms")
    
    return report_df

# Test với data hiện tại
if 'all_evaluations_adaptive' in locals() and all_evaluations_adaptive:
    # Tạo report cho hybrid method (best performing)
    hybrid_basic_results = all_evaluations_adaptive.get('Hybrid + Basic')
    if hybrid_basic_results:
        print("📊 Creating detailed adaptive evaluation report for best method...")
        adaptive_report = updated_create_evaluation_report(hybrid_basic_results, "Hybrid + Basic")
        
        # Save report 
        filename = f"adaptive_evaluation_report_hybrid_basic.csv"
        adaptive_report.to_csv(filename, index=False)
        print(f"\n💾 Adaptive evaluation report saved to '{filename}'")
        
        # Display sample với adaptive metrics
        print(f"\n📋 SAMPLE RESULTS WITH ADAPTIVE METRICS (Top 10 queries):")
        display_cols = ['Query_ID', 'Hit@3', 'MRR', 'Effective_K', 'Adaptive_P@K', 'Response_Time_ms', 
                       'Hit@3_Pass', 'MRR_Pass', 'Adaptive_Precision_Pass']
        print(adaptive_report[display_cols].head(10).to_string(index=False))
    
print("\n✅ Updated create_evaluation_report function with comprehensive adaptive metrics!")
print("🔧 Function: updated_create_evaluation_report()")

📊 Creating detailed adaptive evaluation report for best method...
📋 ADAPTIVE EVALUATION REPORT SUMMARY - Hybrid + Basic
Total Queries Evaluated: 22

📊 ADAPTIVE P@K DISTRIBUTION:
   P@2: 3 queries (13.6%) - Avg: 66.7%
   P@3: 19 queries (86.4%) - Avg: 86.0%

📈 PASS RATES (Adaptive Metrics):
Hit@3 = 100%:      22/22 (100.0%)
MRR ≥ 50%:         21/22 (95.5%)
Adaptive P@K ≥ 70%: 14/22 (63.6%)
Time ≤ 200ms:      20/22 (90.9%)

📊 METRIC STATISTICS (Adaptive):
Hit@3:         Min: 1.00, Max: 1.00, Avg: 1.00
MRR:           Min: 0.333, Max: 1.000, Avg: 0.947
Adaptive P@K:  Min: 0.000, Max: 1.000, Avg: 0.833
Response Time: Min: 78.1ms, Max: 216.1ms, Avg: 132.2ms

💾 Adaptive evaluation report saved to 'adaptive_evaluation_report_hybrid_basic.csv'

📋 SAMPLE RESULTS WITH ADAPTIVE METRICS (Top 10 queries):
 Query_ID  Hit@3  MRR  Effective_K  Adaptive_P@K  Response_Time_ms Hit@3_Pass MRR_Pass Adaptive_Precision_Pass
        1    1.0  1.0            3      1.000000        104.869127          ✅        ✅

In [ ]:
# 📊 TÓM TẮT CÁC CẬP NHẬT ADAPTIVE METRICS
print("="*80)
print("🎯 SUMMARY: ADAPTIVE PRECISION@K IMPLEMENTATION COMPLETED")
print("="*80)

print("\n✅ UPDATED CORE FUNCTIONS:")
print("   1. calculate_precision_at_k() - Uses adaptive K = min(3, num_ground_truth)")
print("   2. evaluate_single_query() - Tracks effective_precision_k")
print("   3. run_full_evaluation() - Maintains compatibility with adaptive metrics")

print("\n✅ UPDATED DISPLAY & ANALYSIS FUNCTIONS:")
print("   4. updated_display_metrics_summary() - Shows adaptive P@K breakdown")
print("   5. updated_compare_methods_with_adaptive_metrics() - Highlights adaptive metrics")
print("   6. updated_analyze_failed_queries() - Analyzes performance by adaptive K")
print("   7. updated_create_evaluation_report() - Comprehensive adaptive reporting")

print("\n✅ NEW COMPREHENSIVE EVALUATION:")
print("   8. run_comprehensive_adaptive_evaluation() - Complete pipeline with adaptive metrics")

print("\n🔧 ADAPTIVE PRECISION@K BENEFITS:")
print("   ✅ Fair evaluation for queries with <3 ground truth documents")
print("   ✅ More accurate performance measurement than fixed P@3")
print("   ✅ Consistent metric calculation across all evaluation functions")
print("   ✅ Detailed breakdown by effective K values")
print("   ✅ Enhanced reporting with adaptive metrics highlighted")

print("\n📈 FINAL PERFORMANCE WITH ADAPTIVE METRICS:")
if 'all_evaluations_adaptive' in locals() and all_evaluations_adaptive:
    best_methods = []
    for method_name, eval_results in all_evaluations_adaptive.items():
        metrics = eval_results['overall_metrics']
        targets = eval_results['target_achievement']
        targets_met = sum([
            targets['hit_at_3_100_percent'] >= 95,
            targets['mrr_above_50_percent'] >= 50,
            targets['response_time_under_200ms'] >= 90
        ])
        
        if targets_met == 3:  # All targets met
            best_methods.append(method_name)
            print(f"   🏆 {method_name}: Hit@3={metrics['hit_at_3_percent']:.1f}%, MRR={metrics['mrr_percent']:.1f}%, Adaptive P@K={metrics['precision_at_3_percent']:.1f}%")
    
    if not best_methods:
        print("   ⚠️  No methods met all 3 targets with adaptive metrics")
else:
    print("   ⚠️  Adaptive evaluation data not available - run evaluation first")

print("\n🎉 ADAPTIVE PRECISION@K IMPLEMENTATION SUCCESSFUL!")
print("   All functions now use consistent, fair adaptive metrics")
print("   System ready for production with accurate performance measurement")
print("="*80)

🎯 SUMMARY: ADAPTIVE PRECISION@K IMPLEMENTATION COMPLETED

✅ UPDATED CORE FUNCTIONS:
   1. calculate_precision_at_k() - Uses adaptive K = min(3, num_ground_truth)
   2. evaluate_single_query() - Tracks effective_precision_k
   3. run_full_evaluation() - Maintains compatibility with adaptive metrics

✅ UPDATED DISPLAY & ANALYSIS FUNCTIONS:
   4. updated_display_metrics_summary() - Shows adaptive P@K breakdown
   5. updated_compare_methods_with_adaptive_metrics() - Highlights adaptive metrics
   6. updated_analyze_failed_queries() - Analyzes performance by adaptive K
   7. updated_create_evaluation_report() - Comprehensive adaptive reporting

✅ NEW COMPREHENSIVE EVALUATION:
   8. run_comprehensive_adaptive_evaluation() - Complete pipeline with adaptive metrics

🔧 ADAPTIVE PRECISION@K BENEFITS:
   ✅ Fair evaluation for queries with <3 ground truth documents
   ✅ More accurate performance measurement than fixed P@3
   ✅ Consistent metric calculation across all evaluation functions
   ✅ Deta

: 